Подготовка букв для распознавания, построение их структурной модели и создание признаковых векторов.

In [ ]:
import os
import copy
import sys
import time
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

import cv2
from collections import deque
import math

from google.colab.patches import cv2_imshow

In [ ]:
from google.colab import drive
drive.mount('\content\gdrive')

In [ ]:
# Функция, которая удаляет с картинки с символом линии, оставшиеся на фоне от
# разлинейки тетрадей.

def thin_picture(img, histI, histJ):
    limit_I = len(histI)
    limit_J = len(histJ)
    for row in range(limit_I):
        if histI[row] >= 17:
            img[row, :] = 255
    for column in range(limit_J):
        if histJ[column] >= 17:
            img[:, column] = 255
    return img

In [ ]:
# Функция построения гистограммы пикселей изображения:
def histogram(img):
    size_i, size_j = img.shape

    pixelsI = np.zeros(size_i)
    pixelsJ = np.zeros(size_j)
    for i in range(size_i):
        for j in range(size_j):
            if img[i, j] == 0:
                pixelsI[i] += 1
                pixelsJ[j] += 1

    return pixelsI, pixelsJ

In [ ]:
def prepare_pictures(img_path):

    img = cv2.imread(img_path)
    bw_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, binary = cv2.threshold(bw_img, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    return bw_img, binary

In [ ]:
# Обозначение соседних для данного пикселя в реализации алгоритма Зонга-Суня:
#     P9 P2 P3
#     P8 P1 P4
#     P7 P6 P5
# Получение соседей пикселя для алгоритма Зонга-Суня:
def neighbours(i, j, img):
    i_1, j_1, i1, j1 = i - 1, j - 1, i + 1, j + 1
    return [ img[i_1][j], img[i_1][j1], img[i][j1], img[i1][j1],     # P2,P3,P4,P5
                img[i1][j], img[i1][j_1], img[i][j_1], img[i_1][j_1] ]    # P6,P7,P8,P9

In [ ]:
# Расчет количества смен с фонового пикселя на пиксель объекта среди соседей
# рассматриваемомго пикселя, т.е. ищется количество шаблонов (n1, n2) = (0, 1):
def transitions(neighbours):
    n = neighbours + neighbours[0: 1]      # P2, P3, ... , P8, P9, P2
    return sum( (n1, n2) == (0, 1) for n1, n2 in zip(n, n[1:]) )  # (P2,P3), (P3,P4), ... , (P8,P9), (P9,P2)

In [ ]:
# Теперь проведем скелетизацию изображения символа.
# Обозначение соседних для данного пикселя P1 в реализации алгоритма Зонга-Суня:
#     P9 P2 P3
#     P8 P1 P4
#     P7 P6 P5
# Первая стадия скелетизации символа - алгоритмом Зонга-Суня:

def zhangSuen(image):

    # Нормализуем изображение и возьмем его копию для сохранения
    # оригинала:
    Image_Thinned = image.copy() / 255
    changing1 = changing2 = 1  # флажки для отметки проведения изменений

    while changing1 or changing2:  # Проходим по картинке до тех пор, пока в ней
        # Шаг 1                     происходят какие-то изменения.
        changing1 = []
        rows, columns = Image_Thinned.shape
        for x in range(1, rows - 1):
            for y in range(1, columns - 1):
                P2,P3,P4,P5,P6,P7,P8,P9 = n = neighbours(x, y, Image_Thinned)
                if (Image_Thinned[x][y] == 1     and  #  0: Точка
                                                      # P1 пренадлежит символу
                    2 <= sum(n) <= 6   and    # Условие 1: 2<= N(P1) <= 6
                    transitions(n) == 1 and    # Условие 2: S(P1)=1
                    P2 * P4 * P6 == 0  and    # Условие 3
                    P4 * P6 * P8 == 0):         # Условие 4
                    changing1.append((x,y))
        for x, y in changing1:
            Image_Thinned[x][y] = 0
        # Шаг 2
        changing2 = []
        for x in range(1, rows - 1):
            for y in range(1, columns - 1):
                P2,P3,P4,P5,P6,P7,P8,P9 = n = neighbours(x, y, Image_Thinned)
                if (Image_Thinned[x][y] == 1   and        # Условие 0
                    2 <= sum(n) <= 6  and                 # Условие 1
                    transitions(n) == 1 and               # Условие 2
                    P2 * P4 * P8 == 0 and                 # Условие 3
                    P2 * P6 * P8 == 0):                   # Условие 4
                    changing2.append((x,y))
        for x, y in changing2:
            Image_Thinned[x][y] = 0
    return Image_Thinned * 255

In [ ]:
# Перед применением алгоритмов скелетизации Зонга-Суня и Ву-Цая проведем
# предварительную обработку изображения по методу, предложенному в работах
# Хаустова: "Хаустов П. А. Алгоритмы распознавания рукописных символов на основе
# построения структурных моделей / П. А. Хаустов // Компьютерная оптика. – 2017.
# – № 41(1). – С. 67-78. – DOI: 10.18287/2412-6179-2017-41-1-67-78":
def preprocess_char(img):

    rows, columns = img.shape
    processedPic = img.copy() / 255

    for i in range(1, rows - 1):
        for j in range(1, columns - 1):
            # Изображение в алгоритме к этому моменту уже инвертированное
            # в противоположность описанию методики в статье Хаустова, где
            # описание ведется на неинвертированном изображении:
            if img[i][j] == 0:  # 0
                P2, P4, P6, P8 = img[i - 1, j], img[i, j + 1], img[i + 1, j],\
                                img[i, j - 1]
                if [P2, P4, P6, P8] != [0, 0, 0, 0] and \
                    [P2, P4, P6, P8] != [0, 1, 0, 1] and \
                    [P2, P4, P6, P8] != [1, 0, 1, 0] and \
                    [P2, P4, P6, P8] != [1, 1, 1, 1]:
                    processedPic[i][j] = 1

    return processedPic * 255

In [ ]:
# Получим 8 соседей для данного пикселя для алгоритма Ву-Цая:
def get_pixels(x, y, img):

    x_1, y_1, x1, y1 = x - 1, y - 1, x + 1, y + 1
    x2, y2 = x + 2, y + 2
    return [ img[x_1][y], img[x_1][y1], img[x][y1], img[x1][y1],    # P2,P3,P4,P5
             img[x1][y], img[x1][y_1], img[x][y_1], img[x_1][y_1],  # P6,P7,P8,P9
             img[x][y2], img[x2][y] ]    # P10, P11

In [ ]:
# Алгоритм Ву-Цая.
# Обозначение соседей для данного пикселя P1 в реализации алгоритма Ву-Цая:
#     P9   P2   P3
#     P8   P1   P4   P10
#     P7   P6   P5
#          P11

def Wu_Tsai(image):
    "Алгоритм Ву-Цая"

    img = (image.copy() / 255).astype('int8')  # создаем копию изображения

    modification = 1  #  задаем флажок для отслеживания изменений
    while modification:       #  производим итерации до тех пор, пока
        modification = []     # какие-то изменения появляются на картинке.
        sizeI, sizeJ = img.shape

        for x in range(1, sizeI-2):
            for y in range(1, sizeJ-2):
                P2, P3, P4, P5, P6, P7, P8, P9, P10, P11 =\
                                                get_pixels(x, y, img)
                # Если текущий пиксель принадлежит объекту, то последовательно
                # проверяем его окружение на соответствие маскам:
                if img[x][y] == 1:
                    # Проверка пикселей на вертикальной грани:
                    if [P2, P4, P6, P7, P8, P9] == [1, 0, 1, 1, 1, 1] and \
                        (P3 == 1 or P5 == 1):
                        modification.append((x, y))
                    elif [P2, P3, P4, P5, P6, P8, P10] == [1, 1, 1, 1, 1, 0, 1]\
                        and (P7 == 1 or P9 == 1):
                        modification.append((x, y))
                    # Проверка пикселей на горизонтальной грани:
                    elif [P2, P3, P4, P6, P8, P9] == [1, 1, 1, 0, 1, 1] and \
                        (P5 == 1 or P7 == 1):
                        modification.append((x, y))
                    elif [P2, P4, P5, P6, P7, P8, P11] == [0, 1, 1, 1, 1, 1, 1] \
                        and (P3 == 1 or P9 == 1):
                        modification.append((x, y))
                    # Проверка пикселей на главной диагонали:
                    elif [P2, P3, P4, P6, P8] == [0, 0, 0, 1, 1] or \
                         [P2, P3, P4, P6, P7, P8] == [1, 1, 1, 0, 0, 0] or \
                         [P2, P3, P4, P5, P6, P7, P8, P9] == \
                                         [1, 0, 1, 0, 0, 0, 0, 0]:
                        modification.append((x, y))
                    # Проверка пикселей на побочной диагонали:
                    elif [P2, P4, P5, P6, P8] == [1, 0, 0, 0, 1] or \
                         [P2, P4, P5, P6, P8, P9] == [0, 1, 1, 1, 0, 0] or \
                         [P2, P3, P4, P5, P6, P7, P8, P9] == \
                                         [0, 0, 1, 0, 1, 0, 0, 0]:
                        modification.append((x, y))
                    # Устранение пикселей шума:
                    elif [P2, P3, P4, P5, P6, P7, P8, P9] == \
                             [0, 0, 0, 1, 1, 1, 0, 0] or \
                         [P2, P3, P4, P5, P6, P7, P8, P9] == \
                             [0, 0, 0, 0, 0, 1, 1, 1] or \
                         [P2, P3, P4, P5, P6, P7, P8, P9] == \
                             [1, 1, 0, 0, 0, 0, 0, 1] or \
                         [P2, P3, P4, P5, P6, P7, P8, P9] == \
                             [0, 1, 1, 1, 0, 0, 0, 0]:
                        modification.append((x, y))

        # Удаляем найденные пиксели из объекта:
        for x, y in modification:
            img[x][y] = 0

    return img * 255

In [ ]:
def alpha_template(neighbours):
    # print(f'Alpha template: {neighbours}')
    return neighbours == [0, 1, 0, 0, 0, 0, 0, 1] or \
           neighbours == [0, 1, 0, 1, 0, 0, 0, 0] or \
           neighbours == [0, 0, 0, 1, 0, 1, 0, 0] or \
           neighbours == [0, 0, 0, 0, 0, 1, 0, 1]

In [ ]:
# Функция проверки соседей пикселя на соответствие первому кругу бета-шаблона.

def centerBetaTemplate(neighbours):
    # print(f'Beta template: {neighbours}')
    return neighbours == [1, 0, 0, 1, 0, 0, 0, 0] or \
           neighbours == [1, 0, 0, 0, 0, 1, 0, 0] or \
           neighbours == [0, 0, 1, 0, 0, 1, 0, 0] or \
           neighbours == [0, 0, 1, 0, 0, 0, 0, 1] or \
           neighbours == [0, 0, 0, 0, 1, 0, 0, 1] or \
           neighbours == [0, 1, 0, 0, 1, 0, 0, 0] or \
           neighbours == [0, 1, 0, 0, 0, 0, 1, 0] or \
           neighbours == [0, 0, 0, 1, 0, 0, 1, 0]

In [ ]:
# Проверка пикселей в качестве кандидата
# для внесения в список точек старта волны в алгоритме Ли.

def get_wave_sources(img, samples_to_delete):

    # Подготовим изображение для обработки:
    img = (img / 255).astype('int16')

    # Заведем пустой список для внесения в него кортежей с координатами точек,
    # которые являются конечными в скелетезированном представлении символа либо
    # имеют больше 2-х инцидентных ребер.
    sources = []
    sizeI, sizeJ = img.shape

    for i in range(sizeI):
        for j in range(sizeJ):
            if img[i][j] == 1:
                if i > 0 and i < (sizeI - 1) and j > 0 and j < (sizeJ - 1):
                    P2, P3, P4, P5, P6, P7, P8, P9 = pixels = neighbours(i, j, img)
                    # Проверим пиксели, лежащие внутри картинки, на соответствие
                    # шаблонам альфа-группы и центарльному кругу бета-группы:
                    if alpha_template(pixels):
                        sources.append((i, j))
                    elif centerBetaTemplate(pixels):
                        sources.append((i, j))
                # В случае, когда пиксель находится в углу, он может быть
                # также одиночным, без какой-либо черты, идущей вглубь
                # картинки. Но это проверять здесь не будем - даже если
                # пиксель в углу одиночный, то алгоритм Ли обнаружит,
                # что из него нет хода.
                elif i > 0 and i < (sizeI - 1) and j == 0:
                    P7 = P8 = P9 = 0
                    P2, P3, P4, P5, P6 = img[i - 1][j], img[i - 1][j + 1], \
                                          img[i][j + 1], img[i + 1][j + 1], \
                                          img[i + 1][j]
                elif i > 0 and i < (sizeI - 1) and j == (sizeJ - 1):
                    P3 = P4 = P5 = 0
                    P6, P7, P8, P9, P2 = img[i + 1][j], img[i + 1][j - 1], \
                                          img[i][j - 1], img[i - 1][j - 1], \
                                          img[i - 1][j]
                elif i == 0 and j > 0 and j < (sizeJ - 1):
                    P9 = P2 = P3 = 0
                    P8, P7, P6, P5, P4 = img[i][j - 1], img[i + 1][j - 1], \
                                          img[i + 1][j], img[i + 1][j + 1], \
                                          img[i][j + 1]
                elif i == (sizeI - 1) and j > 0 and j < (sizeJ - 1):
                    P5 = P6 = P7 = 0
                    P8, P9, P2, P3, P4 = img[i][j - 1], img[i - 1][j - 1], \
                                          img[i - 1][j], img[i - 1][j + 1], \
                                          img[i][j + 1]
                elif i == 0 and j == 0:
                    P2 = P3 = P7 = P8 = P9 = 0
                    P4, P5, P6 = img[i][j + 1], img[i + 1][j + 1], img[i + 1][j]
                elif i == 0 and j == sizeJ - 1:
                    P2 = P3 = P4 = P5 = P9 = 0
                    P6, P7, P8 = img[i + 1][j], img[i + 1][j - 1], img[i][j - 1]
                        #     P9 P2 P3
                        #     P8 P1 P4
                        #     P7 P6 P5
                elif i == sizeI - 1 and j == 0:
                    P5 = P6 = P7 = P8 = P9 = 0
                    P2, P3, P4 = img[i - 1][j], img[i - 1][j + 1], img[i][j + 1]
                elif i == sizeI - 1 and j == sizeJ - 1:
                    P3 = P4 = P5 = P6 = P7 = 0
                    P8, P9, P2 = img[i][j - 1], img[i - 1][j - 1], img[i - 1][j]

                if sum([P2, P3, P4, P5, P6, P7, P8, P9]) == 1 \
                        or (sum([P2, P3, P4, P5, P6, P7, P8, P9]) > 2 and \
                                     (P2 + P3 < 2) and (P3 + P4 < 2) and \
                                     (P4 + P5 < 2) and (P5 + P6 < 2) and \
                                     (P6 + P7 < 2) and (P7 + P8 < 2) and \
                                     (P8 + P9 < 2) and (P9 + P2 < 2)):
                    sources.append((i, j))
                elif sum([P2, P3, P4, P5, P6, P7, P8, P9]) == 0:
                    samples_to_delete.append((i, j))

    return sources, samples_to_delete

In [ ]:
# Функция получения окружения данного пикселя для алгоритма Ли обхода
# бинаризованного скелетизированного начертания символа.
#
# Обозначение соседей данного пикселя в реализации алгоритма обхода графического
# изображения символа для проверки этого пикселя на изгиб:
#            P10
#       P9   P2   P3
# P13   P8   P1   P4   P11
#       P7   P6   P5
#            P12

def get_apex_pixels(x, y, img):

    """Функция возвращает из окружения данного пикселя, которое показано выше,
    пиксели P2, P3, P4, P5, P6, P7, P8, P9."""

    x_1, y_1, x1, y1 = x - 1, y - 1, x + 1, y + 1

    return [ img[x_1][y], img[x_1][y1], img[x][y1], img[x1][y1],    # P2,P3,P4,P5
             img[x1][y], img[x1][y_1], img[x][y_1], img[x_1][y_1]]  # P6,P7,P8,P9

In [ ]:
# Функция проверки возможности сделать шаг на этот пиксель в алгоритме обхода:

def cell_validate(img, i, j):
    return (i >= 0) and (i < len(img)) and (j >= 0) and (j < len(img[0])) \
        and img[i][j] == 1

In [ ]:
# Алгоритм Ли, запускающий одновременно волны из всех точек списка sources.
# В этом первом алгоритме Ли мы находим все фигуры, изображенные на картинке и
# ставим им в соответствие те волны, которые запускались в каждой из них и которые
# обошли контур отдельной фигуры и встретились. Помимом этого мы находим также
# вершины, соответствующие альфа-, бета- и гамма-группам, т.е. те вершины,
# которые являются либо ключевыми вершинами, либо точками изгиба. Пиксели, которые
# имеют петли или имеют больше 2-х инцидентных ребер, т.е. которые, очевидно,
# являются ключевыми узлами, такдже вносятся в список ключевых.

def get_key_nodes(matrix, sources):

    # Нормализуем изображение:
    matrix = (matrix / 255).astype('int16')
    sizeI, sizeJ = matrix.shape

    # Заведем пустое множество, в которое будем складывать вершины, являющиеся
    # ключевыми узлами:
    key_nodes = set()
    # Заведем пустое множество, в которое будем складывать вершины, являющиеся
    # спорными - либо ключевыми, либо перегибами:
    nodes_to_check = set()

    # Создадим для запоминания посещенных пикселей матрицу того же размера, что
    # и матрица, содержащая значения пикселей изображения:
    visited = np.zeros_like(matrix)
    dq = deque()  # Очередь обрабатываемых пикселей.

    # Все возможные перемещения из рассматриваемого пикселя описываются
    # следующими изменениями номеров строки и столбца:
    row = [-1, -1, 0, 1, 1, 1, 0, -1]
    col = [0, 1, 1, 1, 0, -1, -1, -1]

    limit = len(sources)
    for n in range(limit):
        # Считываем координаты очередного источника волны, добавим к координатам
        # пикселя дистанцию от него до источника волны, т.е. 0, добавим номер
        # волны, начиная с 1. При этом волны, запущенные из одного источника
        # будут иметь одинаковые номера. Как можно заметить, номера волн на 1
        # больше номеров источников, из которых испущены эти волны, т.е.
        # идентифицировать источник волны можно по ее номеру. Поставим этот пиксель
        # в очередь для обработки:
        i, j = sources[n]
        dq.append((i, j, 0, n + 1))

        # Отметим пиксель источника волны как посещенный номером волны, которая
        # дошла до этого пикселя, т.е. в этом случае номером самого этого
        # источника:
        visited[i][j] = n + 1

    # Заведем список, в который занесем все фигуры, которые будут идентифицированы
    # на изображении. В этот список будем вставлять кортежи с номерами волн,
    # которые встретились, а значит, они распротсранялись по какой-то одной из
    # фигур на изображении. Отдельно также заведем словарь, в который для каждой
    # волны будем хранить множество всех узлов (и ключевых, и спорных), которые
    # встретились этой волне при распространении:
    samples = []
    node_sequences = {}

    while dq:
        # Достанем слева из очереди узел и обработаем его
        (i, j, dist, wave) = dq.popleft()
        if wave not in node_sequences:
            node_sequences[wave] = set()

        # Проверяем все 8 направлений из данного пикселя и добавляем все возможные
        # из них в очередь справа. При этом, если этот узел находится на краю
        # объекта либо на краю изображения, откуда некуда продолжать движение, то
        # этот пиксель должен быть помечен как ключевой узел. Если же из этого
        # пикселя продолжается объект, но в следующем пикселе уже побывала какая-то
        # другая волна, то все равно этот пиксель надо проверять на изгиб. Для
        # этого заведем счетчик, который будем увеличивать в случае нахождения
        # очередной возможности сделать шаг. Этот счетчик должен быть больше
        # двух, при продолжении объекта дальше с учетом пути, по которому в данный
        # пиксель пришла данная волна:
        move_on = 0
        pixel_sequence = 0
        P2_P9_sequence = 0 # Отдельно проверка на последовательность из пикселей,
                            # являющихся концевыми в круге обхода.
        wave_tale = 0 # Счетчик пикселей, имеющих тот же самый номер волны, что и
                      # данная волна. Он нужен для того, чтобы отличить "хвост"
                      # данной волны от волны вновь встретившейся. Если этот
                      # счетчик станет больше 1-ого после просмотра всех окружающих
                      # данный пиксель пикселей, то значит, несколько волн от одного
                      # и того же источника встретились. Хвостов у волны не может
                      # быть больше одного.

        for k in range(8):

            i_next = i + row[k]
            j_next = j + col[k]

            # Проверим возможность сделать шаг в данном направлении (i_next, j_next):
            if cell_validate(matrix, i_next, j_next):

                move_on += 1
                pixel_sequence += 1
                if k == 0 or k == 7:
                    P2_P9_sequence += 1

                if visited[i_next][j_next] == 0:
                    # Пометим следующий узел как посещенный номером дошедшей
                    # до него волны и добавим его в очередь на обработку справа:
                    visited[i_next][j_next] = wave
                    dq.append((i_next, j_next, dist+1, wave))
                else:
                    visitor = visited[i_next][j_next]
                    # Добавляем к списку ключевых узлов узел, имеющий петлю:
                    if visitor == wave:
                        wave_tale += 1
                    # Установим флажки и найдем в списке samples множества волн,
                    # которые встретили на своем пути волны visitor и wave и
                    # объединим эти множества в одно:
                    visitor_set = wave_set = -1
                    limit = len(samples)
                    for n in range(limit):
                        if visitor in samples[n]:
                            samples[n].add(wave)
                            visitor_set = n
                        elif wave in samples[n]:
                            samples[n].add(visitor)
                            wave_set = n
                    if visitor_set != -1 and wave_set != -1 \
                        and visitor_set != wave_set:
                        samples[visitor_set].update(samples.pop(wave_set))
                    if visitor_set == -1 and wave_set == -1:
                        samples.append(set([wave, visitor]))
            elif pixel_sequence == 1:
                pixel_sequence = 0

        if wave_tale > 1:
            key_nodes.add(sources[wave-1])
            node_sequences[wave].add(sources[wave-1])

        pixel_sequence = max(pixel_sequence, P2_P9_sequence)

        # Если вокруг данного пикселя нашелся только один пиксель, принадлежащий
        # объекту и который соответственно лежит на пути, по которому данная волна
        # пришла в данный пиксель, то этот пиксель находится либо на конце линии
        # объекта, либо на границе изображения. Также, если количество путей
        # вокруг данного пикселя больше двух, тогда этот пиксель является узлом,
        # в котором соединяются больше двух инцидентных ребер. Единственное: в
        # этом случае надо сделать проверку на то, чтобы пиксели объекта вокруг
        # данного пикселя не стояли рядом. Тогда заносим этот пиксель также в
        # список ключевых узлов и заканчиваем эту итерацию цикла по очереди
        # обрабатываемых пикселей dq:
        if move_on < 2:
            key_nodes.add((i, j))
            node_sequences[wave].add((i, j))
            continue
        elif move_on > 2 and pixel_sequence < 2:
            # Проверка на наличие рядом стоящих ключевых узлов:
            if ((i - 1, j) not in key_nodes) and \
                ((i + 1, j) not in key_nodes) and \
                ((i, j - 1) not in key_nodes) and \
                ((i, j + 1) not in key_nodes) and \
                ((i - 1, j - 1) not in key_nodes) and \
                ((i - 1, j + 1) not in key_nodes) and \
                ((i + 1, j - 1) not in key_nodes) and \
                ((i + 1, j + 1) not in key_nodes):
                key_nodes.add((i, j))
                node_sequences[wave].add((i, j))
                continue

        # Если же возможных пикселей вокруг данного пикселя ровно 2, то этот
        # пиксель находится на соединяющем ребре, и его нужно проверять, является
        # ли этот узел перегибом или ключевым, с помощью направляющих векторов.
        # Тогда проверим окружение данного пикселя на соответствие шаблонам альфа-,
        # бета- и гамма-групп. Если это окружение соответствует одному из этих
        # шаблонов, то этот узел нужно добавить в список nodes_to_check для
        # дальнейшей его проверки с помощью направляющих векторов. Для этого
        # необходимо получить соседей данного пикселя:
        #            P10
        #       P9   P2   P3
        # P13   P8   P1   P4   P11
        #       P7   P6   P5
        #            P12
        # При этом, если рассматриваемый пиксель, который принадлежит объекту
        # изображения, расположен от обеих смежных границ изображения ближе, чем на 2
        # пикселя, то он автоматически должен рассматриваться как ключевая точка,
        # потому что в этом случае, даже если объект не прерывается в углу
        # изображения, то направляющие вектора, выходящие из данного пикселя,
        # очевидно, будут иметь угол между собой, который меньше 120 градусов.
        # Если же объект прерывается в углу изображения, то данный пиксель или
        # пиксель в самом углу (если на следующей итерации возможно сделать туда
        # шаг) должен быть отмечен как ключевой.
        if i > 0 and i < sizeI - 1 and j > 0 and j < sizeJ - 1:
            P2, P3, P4, P5, P6, P7, P8, P9 = neighbours = get_apex_pixels(i, j, matrix)
        # Проверим окуружение пикселя на соответствие альфа-шаблону:
        if i > 0 and i < sizeI - 1 and j > 0 and j < sizeJ - 1:
            # Случай, когда пиксель находится на расстоянии 1 пикселя от
            # двух смежных границ изображения. В этом случае, даже если из этого
            # места объект продолжается, но направляющие векторы, выходящие из этого
            # узла, образуют угол меньше 120 градусов. Единственное исключение
            # представляет случай, когда один из соседних пикселей находится в самом
            # углу. В этом случае данный пиксель не должен помечаться как ключевой,
            # а ключевым должен быть помечен угловой пиксель:
            if i == 1 and j == 1 and \
                ([P2, P3, P4, P5, P6, P7, P8, P9] == \
                [0, 1, 0, 1, 0, 0, 0, 0] or \
                [P2, P3, P4, P5, P6, P7, P8, P9] == \
                [0, 0, 0, 1, 0, 1, 0, 0]):
                key_nodes.add((i, j))
                node_sequences[wave].add((i, j))
            elif i == 1 and j == sizeJ - 2 and \
                ([P2, P3, P4, P5, P6, P7, P8, P9] == \
                [0, 0, 0, 1, 0, 1, 0, 0] or \
                [P2, P3, P4, P5, P6, P7, P8, P9] == \
                [0, 0, 0, 0, 0, 1, 0, 1]):
                key_nodes.add((i, j))
                node_sequences[wave].add((i, j))
            elif i == sizeI - 2 and j == 1 and \
                ([P2, P3, P4, P5, P6, P7, P8, P9] == \
                [0, 1, 0, 0, 0, 0, 0, 1] or \
                [P2, P3, P4, P5, P6, P7, P8, P9] == \
                [0, 1, 0, 1, 0, 0, 0, 0]):
                key_nodes.add((i, j))
                node_sequences[wave].add((i, j))
            elif i == sizeI - 2 and j == sizeJ - 2 and \
                ([P2, P3, P4, P5, P6, P7, P8, P9] == \
                [0, 1, 0, 0, 0, 0, 0, 1] or \
                [P2, P3, P4, P5, P6, P7, P8, P9] == \
                [0, 0, 0, 0, 0, 1, 0, 1]):
                key_nodes.add((i, j))
                node_sequences[wave].add((i, j))
            elif alpha_template(neighbours):
                # print(f'Alpha_template сработал, соседи пикселя {(i, j)}:'
                                        # f'{[P2, P3, P4, P5, P6, P7, P8, P9]}')
                nodes_to_check.add((i, j))
                node_sequences[wave].add((i, j))
                continue
        # Далее проверяем соседей пикселя на соответствие бета- и гамма-шаблонам
        # и проверяем ситуацию нахождения пикселя в одном из углов изображения.
        # Первый случай - это пиксель находится на расстоянии, равном или большем
        # двух пикселей от всех границ изображения:
        # Условие 1
        if i > 1 and i < sizeI - 2 and j > 1 and j < sizeJ - 2:
            P10, P11, P12, P13 = matrix[i - 2][j], matrix[i][j + 2], \
                                  matrix[i + 2][j], matrix[i][j - 2]
            # # Проверка окружения пикселя на соответствие шаблону альфа-группы:
            # if alpha_template(neighbours):
            #     print(f'Условие 1, alpha_template сработал')
            #     nodes_to_check.append((i, j))
            # Проверка окружения пикселя на соответствие шаблону бета-группы:
            if [P2, P3, P4, P5, P6, P7, P8, P9, P11] == \
                [1, 0, 0, 1, 0, 0, 0, 0, 1] or \
                [P2, P3, P4, P5, P6, P7, P8, P9, P13] == \
                [1, 0, 0, 0, 0, 1, 0, 0, 1] or \
                [P2, P3, P4, P5, P6, P7, P8, P9, P12] == \
                [0, 0, 1, 0, 0, 1, 0, 0, 1] or \
                [P2, P3, P4, P5, P6, P7, P8, P9, P10] == \
                [0, 0, 1, 0, 0, 0, 0, 1, 1] or \
                [P2, P3, P4, P5, P6, P7, P8, P9, P13] == \
                [0, 0, 0, 0, 1, 0, 0, 1, 1] or \
                [P2, P3, P4, P5, P6, P7, P8, P9, P11] == \
                [0, 1, 0, 0, 1, 0, 0, 0, 1] or \
                [P2, P3, P4, P5, P6, P7, P8, P9, P10] == \
                [0, 1, 0, 0, 0, 0, 1, 0, 1] or \
                [P2, P3, P4, P5, P6, P7, P8, P9, P12] == \
                [0, 0, 0, 1, 0, 0, 1, 0, 1]:
                nodes_to_check.add((i, j))
                node_sequences[wave].add((i, j))
            # Проверка окружения пикселя на соответствие первому кругу шаблона
            # бета-группы. Если соседи пикселя соответствуют этому шаблону, тогда
            # вносим пиксель в список узлов, которые нужно дополнительно
            # проверить с помощью направляющих векторов, выходящих из него:
            elif centerBetaTemplate(neighbours):
                nodes_to_check.add((i, j))
                node_sequences[wave].add((i, j))
            # Проверка окружения пикселя на соответствие шаблону гамма-группы:
            elif [P2, P3, P4, P5, P6, P7, P8, P9, P10, P11] == \
                [0, 0, 0, 1, 0, 0, 0, 1, 1, 1] or \
                [P2, P3, P4, P5, P6, P7, P8, P9, P12, P13] == \
                [0, 0, 0, 1, 0, 0, 0, 1, 1, 1] or \
                [P2, P3, P4, P5, P6, P7, P8, P9, P10, P13] == \
                [0, 1, 0, 0, 0, 1, 0, 0, 1, 1] or \
                [P2, P3, P4, P5, P6, P7, P8, P9, P11, P12] == \
                [0, 1, 0, 0, 0, 1, 0, 0, 1, 1] or \
                [P2, P3, P4, P5, P6, P7, P8, P9, P10, P12] == \
                [0, 0, 0, 1, 0, 0, 0, 1, 1, 1] or \
                [P2, P3, P4, P5, P6, P7, P8, P9, P11, P13] == \
                [0, 0, 0, 1, 0, 0, 0, 1, 1, 1] or \
                [P2, P3, P4, P5, P6, P7, P8, P9, P10, P12] == \
                [0, 1, 0, 0, 0, 1, 0, 0, 1, 1] or \
                [P2, P3, P4, P5, P6, P7, P8, P9, P11, P13] == \
                [0, 1, 0, 0, 0, 1, 0, 0, 1, 1]:
                nodes_to_check.add((i, j))
                node_sequences[wave].add((i, j))
        # Случай, когда пиксель находится на расстоянии 1 пикселя от
        # верхней границы изображения, но на расстоянии, большем или равном двум
        # пикселям, от правой и левой границ изображения:
        # Условие 2
        elif i == 1 and j > 1 and j < sizeJ - 2:
            P11, P12, P13 = matrix[i][j + 2], matrix[i + 2][j], matrix[i][j - 2]
            # # Проверка окружения пикселя на соответствие шаблону альфа-группы:
            # if alpha_template(neighbours):
            #     print(f'Условие 2, alpha_template сработал')
            #     nodes_to_check.append((i, j))
            # Проверка окружения пикселя на соответствие шаблону бета-группы:
            if [P2, P3, P4, P5, P6, P7, P8, P9, P11] == \
                [1, 0, 0, 1, 0, 0, 0, 0, 1] or \
                [P2, P3, P4, P5, P6, P7, P8, P9, P13] == \
                [1, 0, 0, 0, 0, 1, 0, 0, 1] or \
                [P2, P3, P4, P5, P6, P7, P8, P9, P12] == \
                [0, 0, 1, 0, 0, 1, 0, 0, 1] or \
                [P2, P3, P4, P5, P6, P7, P8, P9, P13] == \
                [0, 0, 0, 0, 1, 0, 0, 1, 1] or \
                [P2, P3, P4, P5, P6, P7, P8, P9, P11] == \
                [0, 1, 0, 0, 1, 0, 0, 0, 1] or \
                [P2, P3, P4, P5, P6, P7, P8, P9, P12] == \
                [0, 0, 0, 1, 0, 0, 1, 0, 1]:
                nodes_to_check.add((i, j))
                node_sequences[wave].add((i, j))
            # Проверка окружения пикселя на соответствие первому кругу шаблона
            # бета-группы. Если соседи пикселя соответствуют этому шаблону, тогда
            # вносим пиксель в список узлов, которые нужно дополнительно
            # проверить с помощью направляющих векторов, выходящих из него:
            elif centerBetaTemplate(neighbours):
                # print(f'Условие 2, centerBetaTemplate сработал')
                nodes_to_check.add((i, j))
                node_sequences[wave].add((i, j))
            # Проверка окружения пикселя на соответствие шаблону гамма-группы:
            elif [P2, P3, P4, P5, P6, P7, P8, P9, P12, P13] == \
                [0, 0, 0, 1, 0, 0, 0, 1, 1, 1] or \
                [P2, P3, P4, P5, P6, P7, P8, P9, P11, P12] == \
                [0, 1, 0, 0, 0, 1, 0, 0, 1, 1] or \
                [P2, P3, P4, P5, P6, P7, P8, P9, P11, P13] == \
                [0, 0, 0, 1, 0, 0, 0, 1, 1, 1] or \
                [P2, P3, P4, P5, P6, P7, P8, P9, P11, P13] == \
                [0, 1, 0, 0, 0, 1, 0, 0, 1, 1]:
                nodes_to_check.add((i, j))
                node_sequences[wave].add((i, j))
        # Случай, когда пиксель находится на расстоянии 1 пикселя от
        # нижней границы изображения, но на расстоянии, большем или равном двум
        # пикселям, от правой и левой границ изображения:
        # Условие 3
        elif i == sizeI - 2 and j > 1 and j < sizeJ - 2:
            P10, P11, P13 = matrix[i - 2][j], matrix[i][j + 2], matrix[i][j - 2]
            # # Проверка окружения пикселя на соответствие шаблону альфа-группы:
            # if alpha_template(neighbours):
            #     print(f'Условие 3, alpha_template сработал')
            #     nodes_to_check.append((i, j))
            # Проверка окружения пикселя на соответствие шаблону бета-группы:
            if [P2, P3, P4, P5, P6, P7, P8, P9, P11] == \
                [1, 0, 0, 1, 0, 0, 0, 0, 1] or \
                [P2, P3, P4, P5, P6, P7, P8, P9, P13] == \
                [1, 0, 0, 0, 0, 1, 0, 0, 1] or \
                [P2, P3, P4, P5, P6, P7, P8, P9, P10] == \
                [0, 0, 1, 0, 0, 0, 0, 1, 1] or \
                [P2, P3, P4, P5, P6, P7, P8, P9, P13] == \
                [0, 0, 0, 0, 1, 0, 0, 1, 1] or \
                [P2, P3, P4, P5, P6, P7, P8, P9, P11] == \
                [0, 1, 0, 0, 1, 0, 0, 0, 1] or \
                [P2, P3, P4, P5, P6, P7, P8, P9, P10] == \
                [0, 1, 0, 0, 0, 0, 1, 0, 1]:
                nodes_to_check.add((i, j))
                node_sequences[wave].add((i, j))
            # Проверка окружения пикселя на соответствие первому кругу шаблона
            # бета-группы. Если соседи пикселя соответствуют этому шаблону, тогда
            # вносим пиксель в список узлов, которые нужно дополнительно
            # проверить с помощью направляющих векторов, выходящих из него:
            elif centerBetaTemplate(neighbours):
                # print(f'Условие 3, centerBetaTemplate сработал')
                nodes_to_check.add((i, j))
                node_sequences[wave].add((i, j))
            # Проверка окружения пикселя на соответствие шаблону гамма-группы:
            elif [P2, P3, P4, P5, P6, P7, P8, P9, P10, P11] == \
                [0, 0, 0, 1, 0, 0, 0, 1, 1, 1] or \
                [P2, P3, P4, P5, P6, P7, P8, P9, P10, P13] == \
                [0, 1, 0, 0, 0, 1, 0, 0, 1, 1] or \
                [P2, P3, P4, P5, P6, P7, P8, P9, P11, P13] == \
                [0, 0, 0, 1, 0, 0, 0, 1, 1, 1] or \
                [P2, P3, P4, P5, P6, P7, P8, P9, P11, P13] == \
                [0, 1, 0, 0, 0, 1, 0, 0, 1, 1]:
                nodes_to_check.add((i, j))
                node_sequences[wave].add((i, j))
        # Случай, когда пиксель находится на расстоянии 1 пикселя от левой
        # границы изображения, но на расстоянии, большем или равном двум
        # пикселям, от верхней и нижней границ изображения:
        # Условие 4
        elif i > 1 and i < sizeI - 2 and j == 1:
            P10, P11, P12 = matrix[i - 2][j], matrix[i][j + 2], matrix[i + 2][j]
            # # Проверка окружения пикселя на соответствие шаблону альфа-группы:
            # if alpha_template(neighbours):
            #     print(f'Условие 4, alpha_template сработал')
            #     nodes_to_check.append((i, j))
            # Проверка окружения пикселя на соответствие шаблону бета-группы:
            if [P2, P3, P4, P5, P6, P7, P8, P9, P11] == \
                [1, 0, 0, 1, 0, 0, 0, 0, 1] or \
                [P2, P3, P4, P5, P6, P7, P8, P9, P12] == \
                [0, 0, 1, 0, 0, 1, 0, 0, 1] or \
                [P2, P3, P4, P5, P6, P7, P8, P9, P10] == \
                [0, 0, 1, 0, 0, 0, 0, 1, 1] or \
                [P2, P3, P4, P5, P6, P7, P8, P9, P11] == \
                [0, 1, 0, 0, 1, 0, 0, 0, 1] or \
                [P2, P3, P4, P5, P6, P7, P8, P9, P10] == \
                [0, 1, 0, 0, 0, 0, 1, 0, 1] or \
                [P2, P3, P4, P5, P6, P7, P8, P9, P12] == \
                [0, 0, 0, 1, 0, 0, 1, 0, 1]:
                nodes_to_check.add((i, j))
                node_sequences[wave].add((i, j))
            # Проверка окружения пикселя на соответствие первому кругу шаблона
            # бета-группы. Если соседи пикселя соответствуют этому шаблону, тогда
            # вносим пиксель в список узлов, которые нужно дополнительно
            # проверить с помощью направляющих векторов, выходящих из него:
            elif centerBetaTemplate(neighbours):
                # print(f'Условие 4, centerBetaTemplate сработал')
                nodes_to_check.add((i, j))
                node_sequences[wave].add((i, j))
            # Проверка окружения пикселя на соответствие шаблону гамма-группы:
            elif [P2, P3, P4, P5, P6, P7, P8, P9, P10, P11] == \
                [0, 0, 0, 1, 0, 0, 0, 1, 1, 1] or \
                [P2, P3, P4, P5, P6, P7, P8, P9, P11, P12] == \
                [0, 1, 0, 0, 0, 1, 0, 0, 1, 1] or \
                [P2, P3, P4, P5, P6, P7, P8, P9, P10, P12] == \
                [0, 0, 0, 1, 0, 0, 0, 1, 1, 1] or \
                [P2, P3, P4, P5, P6, P7, P8, P9, P10, P12] == \
                [0, 1, 0, 0, 0, 1, 0, 0, 1, 1]:
                nodes_to_check.add((i, j))
                node_sequences[wave].add((i, j))
        # Случай, когда пиксель находится на расстоянии 1 пикселя от правой
        # границы изображения, но на расстоянии, большем или равном двум
        # пикселям, от верхней и нижней границ изображения:
        # Условие 5
        elif i > 1 and i < sizeI - 2 and j == sizeJ - 2:
            P10, P12, P13 = matrix[i - 2][j], matrix[i + 2][j], matrix[i][j - 2]
            # # Проверка окружения пикселя на соответствие шаблону альфа-группы:
            # if alpha_template(neighbours):
            #     print(f'Условие 5, alpha_template сработал')
            #     nodes_to_check.append((i, j))
            # Проверка окружения пикселя на соответствие шаблону бета-группы:
            if [P2, P3, P4, P5, P6, P7, P8, P9, P13] == \
                [1, 0, 0, 0, 0, 1, 0, 0, 1] or \
                [P2, P3, P4, P5, P6, P7, P8, P9, P12] == \
                [0, 0, 1, 0, 0, 1, 0, 0, 1] or \
                [P2, P3, P4, P5, P6, P7, P8, P9, P10] == \
                [0, 0, 1, 0, 0, 0, 0, 1, 1] or \
                [P2, P3, P4, P5, P6, P7, P8, P9, P13] == \
                [0, 0, 0, 0, 1, 0, 0, 1, 1] or \
                [P2, P3, P4, P5, P6, P7, P8, P9, P10] == \
                [0, 1, 0, 0, 0, 0, 1, 0, 1] or \
                [P2, P3, P4, P5, P6, P7, P8, P9, P12] == \
                [0, 0, 0, 1, 0, 0, 1, 0, 1]:
                nodes_to_check.add((i, j))
                node_sequences[wave].add((i, j))
            # Проверка окружения пикселя на соответствие первому кругу шаблона
            # бета-группы. Если соседи пикселя соответствуют этому шаблону, тогда
            # вносим пиксель в список узлов, которые нужно дополнительно
            # проверить с помощью направляющих векторов, выходящих из него:
            elif centerBetaTemplate(neighbours):
                # print(f'Условие 5, centerBetaTemplate сработал')
                nodes_to_check.add((i, j))
                node_sequences[wave].add((i, j))
            # Проверка окружения пикселя на соответствие шаблону гамма-группы:
            elif [P2, P3, P4, P5, P6, P7, P8, P9, P12, P13] == \
                [0, 0, 0, 1, 0, 0, 0, 1, 1, 1] or \
                [P2, P3, P4, P5, P6, P7, P8, P9, P10, P13] == \
                [0, 1, 0, 0, 0, 1, 0, 0, 1, 1] or \
                [P2, P3, P4, P5, P6, P7, P8, P9, P10, P12] == \
                [0, 0, 0, 1, 0, 0, 0, 1, 1, 1] or \
                [P2, P3, P4, P5, P6, P7, P8, P9, P10, P12] == \
                [0, 1, 0, 0, 0, 1, 0, 0, 1, 1]:
                nodes_to_check.add((i, j))
                node_sequences[wave].add((i, j))
        # В случае, если пиксель находится в одном из углов изображения:
        elif (i == 0 and j == 0) or (i == 0 and j == sizeJ - 1) or \
              (i == sizeI - 1 and j == 0) or (i == sizeI - 1 and j == sizeJ - 1):
            key_nodes.add((i, j))
            node_sequences[wave].add((i, j))

    return key_nodes, nodes_to_check, samples, node_sequences

In [ ]:
# Функция удаления с изображения объектов, у которых меньше трех узлов.
# Предположительно, такие объекты не являются интересующими нас символами, потому
# что имеют только два концевых ключевых узла и не имеют узлов разветвления и
# узлов перегиба.

def delete_samples_from_pic(img, sample_nodes):

    img = (img / 255).astype('int16')

    row = [-1, -1, 0, 1, 1, 1, 0, -1]
    col = [0, 1, 1, 1, 0, -1, -1, -1]
    dq = deque()

    for node in sample_nodes:
        dq.append(node)

    while dq:
        i, j = dq.popleft()
        for k in range(8):
            i_next = i + row[k]
            j_next = j + col[k]
            if cell_validate(img, i_next, j_next):
                dq.append((i_next, j_next))
        img[i][j] = 0

    return img * 255

In [ ]:
# Класс, в котором реализуются методы для расчета длин направляющих векторов.

class guiding_vector:

    def __init__(self, node_coordinates, node_id, vector_id, distance):
        self.node_i = node_coordinates[0]
        self.node_j = node_coordinates[1]
        self.node_id = node_id
        self.vector_id = vector_id
        # Параметр дистанции в пикселях, по которому будем отслеживать появление
        # разветвления и который также будет использоваться для расчета вектора:
        self.distance = distance
        # Флаг разветвления, который должен будет поменять очередной
        # обрабатываемый из очереди пиксель, если поле distance уже имеет
        # то же значение, что и у этого пикселя:
        self.branching = 0
        self.coordinates = [0, 0]
        # Предусмотрим возможность отката к предыдущим координатам, если на
        # следующем шаге выяснится, что произошло разветвление:
        self.previous = self.coordinates.copy()
    def update(self, i_new, j_new):
        self.previous = self.coordinates.copy()
        self.coordinates[0] += (i_new - self.node_i)/np.power(2, (self.distance-1),
                                                         dtype = float)
        self.coordinates[1] += (j_new - self.node_j)/np.power(2, (self.distance-1),
                                                         dtype = float)
    def rollback(self):  # Метод отката к координатам предыдущего шага.
        self.coordinates = self.previous.copy()

In [ ]:
# Теперь, имея список спорных пикселей выделим из этого списка пиксели
# перегибов и ключевые пиксели. Для этого мы одновременно запускаем волны из
# всех найденных ключевых пикселей и продсчитываем угол между двумя векторами
# направлений из данного пикселя:
def check_nodes(image, nodes_to_check, key_nodes_dict):

    # Список key_nodes содержит только явные ключевые узлы - узлы, которые имеют
    # либо больше двух инцидентных ребер, либо одно инцидентное ребро, либо имеют
    # петлю. Поэтому в этой функции при расчете векторов узлов, которые будем
    # проверять, создадим для каждого такого узла экземпляр класса node и занесем
    # в него найденные векторы. Для каждого такого вектора сформируем его id в
    # рамках данного узла по тем же правилам, что и раньше: id узла будет
    # соответствовать значению k, определяющему соседний пиксель данного узла, из
    # которого начинается данный вектор. При этом список key_nodes для
    # последующих манипуляций уже должен содержать экземпляры nodes, которые нужно
    # сформировать в основной функции уже до вызова данной функции check_nodes.
    # id ключевых узлов уже сформируем. id узлов изгибов будем продолжать,
    # начиная после последнего ключевого узла. Нумерацию ключевых узлов следует
    # начинать с 1, чтобы в разных манипуляциях не путать помеченные после
    # посещения пиксели с пикселями, которые еще не посещались ни одной волной ни
    # из одного узла. В эту функцию будем передавать номер последнего
    # помеченного ключевого узла. В функции будем создавать список дополнительных
    # ключевых узлов, которые будем возвращать для добавления к уже имеющимся
    # узлам.

    bending_nodes_dict = {} # Заведем словарь для записи в него найденных узлов-перегибов
    additional_key_nodes = {}  #  Этот словарь нам нужен для того, чтобы потом
                                # создать для вновь найденных ключевых узлов
                                # экземпляры Node
    found_new_key_node = 1 # Признак, который будет запускать постоянно цикл
                            # проверки узлов
    node_vectors = {}

    while found_new_key_node:

        additional_node_vectors = node_guide_vectors_calc(image, nodes_to_check,
                                                          key_nodes_dict)
        node_vectors.update(additional_node_vectors)

        additional_key_nodes_dict = {} # Словарь дополнительных ключевых узлов
        found_new_key_node = 0 # Признак, который будет запускать постоянно цикл
                                # проверки узлов, если определен очередной ключевой
                                # узел.

        for node_id, vectors in additional_node_vectors.items():
            vectors_quantity = len(vectors)
            if vectors_quantity < 2 or vectors_quantity > 2:
                key_nodes_dict[node_id] = nodes_to_check[node_id]
                additional_key_nodes[node_id] = nodes_to_check[node_id]
                found_new_key_node = 1
                del(nodes_to_check[node_id])
                continue
            vector_coordinates = []
            for vector in vectors.values():
                vector_coordinates.append(vector.coordinates)
            vector_1 = np.array(vector_coordinates[0], dtype = float)
            vector_2 = np.array(vector_coordinates[1], dtype = float)
                # print(f'Эти вектора: {vector_1} и {vector_2}.')
            norm_1 = np.linalg.norm(vector_1, ord = 2)
            norm_2 = np.linalg.norm(vector_2, ord = 2)
            # print(f'Вычисленные нормы векторов: вектор1: {norm_1}, вектор2: {norm_2}.')
            # print(f'Скалярное произведение этих векторов дает: {np.dot(vector_1, vector_2)}.')
            angle = math.degrees(np.arccos(np.dot(vector_1, vector_2) / (norm_1 * norm_2)))
            # print(f'Угол между этими векторами составил: {angle} градусов.')
            if angle >= 120:
                bending_nodes_dict[node_id] = nodes_to_check[node_id]
                # print(f'Поэтому узел {node} с координатами
                # {nodes_to_check[node]} заносится в список ИЗГИБОВ.')
            else:
                key_nodes_dict[node_id] = nodes_to_check[node_id]
                additional_key_nodes[node_id] = nodes_to_check[node_id]
                found_new_key_node = 1
                # print(f'Поэтому узел {node} с координатами
                # {nodes_to_check[node]} заносится в список КЛЮЧЕВЫХ УЗЛОВ.')
            del(nodes_to_check[node_id])

    # Теперь, после нескольких итераций проверки спорных узлов, пройдемся по
    # словарям additional_key_nodes и bending_nodes_dict
    # и создадим для каждого картежа координат узлов экземпляр класса
    # node. id соответствующих узлов и их векторов будем брать из словаря
    # node_vectors[vc.node_id][vc.vector_id]

    additional_key_nodes = create_nodes(additional_key_nodes, node_vectors)

    bending_nodes = create_nodes(bending_nodes_dict, node_vectors)

    return additional_key_nodes, bending_nodes

In [ ]:
# Функция создает экземпляры узлов класса node по переоданному в нее словарю и
# добавляет каждому узлу его вектора, переданные в словаре node_vectors.

def create_nodes(nodes_dict, node_vectors):
    nodes = []
    for node_id, node_coordinates in nodes_dict.items():
        node_i, node_j = node_coordinates
        new_node = Node(node_i, node_j, node_id)
        for vector_id, vector in node_vectors[node_id].items():
            new_node.new_vector(vector)
        nodes.append(new_node)
    return nodes

In [ ]:
#@title Код, строящий дугу на основе алгоритма Брезенхема.

# Ниже написан скрипт, реализующий алгоритм на основе алгоритма Брезенхама,
# который строит дугу. Взят с:
# https://www.daniweb.com/programming/software-development/threads/321181/python-bresenham-circle-arc-algorithm
#
# Есть так же другой вариант, который, кажется, более интересным:
# https://gist.github.com/DmitriyPoltavskiy/4eb47f7124853940315df4a71a12948e
# Его надо просто перевести на Python.
#
# Довольно короткий код для построения дуги окружности на C++:
# https://codingee.com/computer-graphics-program-to-implement-circular-arc-algorithm/
#
import PIL.Image, PIL.ImageDraw, math, time

def circle_arc(radius, start, end, clockwise=True):
    """Python implementation of the modified Bresenham algorithm
    for complete circles, arcs and pies
    radius: radius of the circle in pixels
    start and end are angles in radians
    function will return a list of points (tuple coordinates)
    and the coordinates of the start and end point in a list xy
    """
    if start>=math.pi*2:
        start = math.radians(math.degrees(start)%360)
    if end>=math.pi*2:
        end = math.radians(math.degrees(end)%360)
    # always clockwise drawing, if anti-clockwise drawing desired
    # exchange start and end
    if not clockwise:
        s = start
        start = end
        end = s
    # determination which quarters and octants are necessary
    # init vars
    xy = [[0,0], [0,0]] # to locate actual start and end point for pies
    # the x/y border value for drawing the points
    q_x = []
    q_y = []
    # first q element in list q is quarter of start angle
    # second q element is quarter of end angle
    # now determine the quarters to compute
    q = []
    # 0 - 90 degrees = 12 o clock to 3 o clock = 0.0 - math.pi/2 --> q==1
    # 90 - 180 degrees = math.pi/2 - math.pi --> q==2
    # 180 - 270 degrees = math.pi - math.pi/2*3 --> q==3
    # 270 - 360 degrees = math.pi/2*3 - math.pi*2 --> q==4
    j = 0
    for i in [start, end]:
        angle = i
        if angle<math.pi/2:
            q.append(1)
            # compute the minimum x and y-axis value for plotting
            q_x.append(int(round(math.sin(angle)*radius)))
            q_y.append(int(round(math.cos(angle)*radius)))
            if j==1 and angle==0:
                xy[1] = [0,-radius] # 90 degrees
        elif angle>=math.pi/2 and angle<math.pi:
            q.append(2)
            # compute the minimum x and y-axis value for plotting
            q_x.append(int(round(math.cos(angle-math.pi/2)*radius)))
            q_y.append(int(round(math.sin(angle-math.pi/2)*radius)))
            if j==1 and angle==math.pi/2:
                xy[1] = [radius,0] # 90 degrees
        elif angle>=math.pi and angle<math.pi/2*3:
            q.append(3)
            # compute the minimum x and y-axis value for plotting
            q_x.append(int(round(math.sin(angle-math.pi)*radius)))
            q_y.append(int(round(math.cos(angle-math.pi)*radius)))
            if j==1 and angle==math.pi:
                xy[1] = [0, radius]
        else:
            q.append(4)
            # compute the minimum x and y-axis value for plotting
            q_x.append(int(round(math.cos(angle-math.pi/2*3)*radius)))
            q_y.append(int(round(math.sin(angle-math.pi/2*3)*radius)))
            if j==1 and angle==math.pi/2*3:
                xy[1] = [-radius, 0]
        j = j + 1
    # print "q", q, "q_x", q_x, "q_y", q_y
    quarters = []
    sq = q[0]
    while 1:
        quarters.append(sq)
        if q[1] == sq and start<end or q[1] == sq and start>end and q[0]!=q[1]:
            break # we reach the final end quarter
        elif q[1] == sq and start>end:
            quarters.extend([(sq+1)%5, (sq+2)%5, (sq+3)%5, (sq+4)%5])
            break
        else:
            sq = sq + 1
            if sq>4:
                sq = 1
    # print "quarters", quarters
    switch = 3 - (2 * radius)
    points = []
    points1 = set()
    points2 = set()
    points3 = set()
    points4 = set()
    #
    x = 0
    y = radius
    # first quarter/octant starts clockwise at 12 o'clock
    while x <= y:
        if 1 in quarters:
            if not (1 in q):
                # add all points of the quarter completely
                # first quarter first octant
                points1.add((x,-y))
                # first quarter 2nd octant
                points1.add((y,-x))
            else:
                # start or end point in this quarter?
                if q[0] == 1:
                    # start point
                    if q_x[0]<=x and q_y[0]>=abs(-y) and len(quarters)>1 or q_x[0]<=x and q_x[1]>=x:
                        # first quarter first octant
                        points1.add((x,-y))
                        if -y<xy[0][1]:
                            xy[0] = [x,-y]
                        elif -y==xy[0][1]:
                            if x<xy[0][0]:
                                xy[0] = [x,-y]
                    if q_x[0]<=y and q_y[0]>=abs(-x) and len(quarters)>1 or q_x[0]<=y and q_x[1]>=y and q_y[0]>=abs(-x) and q_y[1]<=abs(-x):
                        # first quarter 2nd octant
                        points1.add((y,-x))
                        if -x<xy[0][1]:
                            xy[0] = [y,-x]
                        elif -x==xy[0][1]:
                            if y<xy[0][0]:
                                xy[0] = [y,-x]
                if q[1] == 1:
                    # end point
                    if q_x[1]>=x and len(quarters)>1 or q_x[0]<=x and q_x[1]>=x:
                        # first quarter first octant
                        points1.add((x,-y))
                        if x>xy[1][0]:
                            xy[1] = [x,-y]
                        elif x==xy[1][0]:
                            if -y>xy[1][1]:
                                xy[1] = [x,-y]
                    if q_x[1]>=y and q_y[1]<=abs(-x) and len(quarters)>1 or q_x[0]<=y and q_x[1]>=y and q_y[0]>=abs(-x) and q_y[1]<=abs(-x):
                        # first quarter 2nd octant
                        points1.add((y,-x))
                        if y>xy[1][0]:
                            xy[1] = [y,-x]
                        elif y==xy[1][0]:
                            if -x>xy[1][1]:
                                xy[1] = [y,-x]
        if 2 in quarters:
            if not (2 in q):
                # add all points of the quarter completely
                # second quarter 3rd octant
                points2.add((y,x))
                # second quarter 4.octant
                points2.add((x,y))
            else:
                # start or end point in this quarter?
                if q[0] == 2:
                    # start point
                    if q_x[0]>=y and q_y[0]<=x and len(quarters)>1 or q_x[0]>=y and q_x[1]<=y and q_y[0]<=x and q_y[1]>=x:
                        # second quarter 3rd octant
                        points2.add((y,x))
                        if y>xy[0][0]:
                            xy[0] = [y,x]
                        elif y==xy[0][0]:
                            if x<xy[0][1]:
                                xy[0] = [y,x]
                    if q_x[0]>=x and q_y[0]<=y and len(quarters)>1 or q_x[0]>=x and q_x[1]<=x:
                        # second quarter 4.octant
                        points2.add((x,y))
                        if x>xy[0][0]:
                            xy[0] = [x,y]
                        elif x==xy[0][0]:
                            if y<xy[0][1]:
                                xy[0] = [x,y]
                if q[1] == 2:
                    # end point
                    if q_x[1]<=y and q_y[1]>=x and len(quarters)>1 or q_x[0]>=y and q_x[1]<=y and q_y[0]<=x and q_y[1]>=x:
                        # second quarter 3rd octant
                        points2.add((y,x))
                        if x>xy[1][1]:
                            xy[1] = [y,x]
                        elif x==xy[1][1]:
                            if y<xy[1][0]:
                                xy[1] = [y,x]
                    if q_x[1]<=x and q_y[1]>=y and len(quarters)>1 or q_x[0]>=x and q_x[1]<=x:
                        # second quarter 4.octant
                        points2.add((x,y))
                        if y>xy[1][1]:
                            xy[1] = [x,y]
                        elif x==xy[1][1]:
                            if x<xy[1][0]:
                                xy[1] = [x,y]
        if 3 in quarters:
            if not (3 in q):
                # add all points of the quarter completely
                # third quarter 5.octant
                points3.add((-x,y))
                # third quarter 6.octant
                points3.add((-y,x))
            else:
                # start or end point in this quarter?
                if q[0] == 3:
                    # start point
                    if q_x[0]<=x and q_y[0]>=abs(y) and len(quarters)>1 or q_x[0]<=x and q_x[1]>=x:
                        # third quarter 5.octant
                        points3.add((-x,y))
                        if y>xy[0][1]:
                            xy[0] = [-x,y]
                        elif y==xy[0][1]:
                            if -x>xy[0][0]:
                                xy[0] = [-x,y]
                    if q_x[0]<=y and q_y[0]>=abs(x) and len(quarters)>1 or q_x[0]<=y and q_x[1]>=y and q_y[0]>=x and q_y[1]<=x:
                        # third quarter 6.octant
                        points3.add((-y,x))
                        if x>xy[0][1]:
                            xy[0] = [-y,x]
                        elif x==xy[0][1]:
                            if -y>xy[0][0]:
                                xy[0] = [-y,x]
                if q[1] == 3:
                    # end point
                    if q_x[1]>=x and len(quarters)>1 or q_x[0]<=x and q_x[1]>=x:
                        # third quarter 5.octant
                        points3.add((-x,y))
                        if -x<xy[1][0]:
                            xy[1] = [-x,y]
                        elif -x==xy[1][0]:
                            if y<xy[1][1]:
                                xy[1] = [-x,y]
                    if q_x[1]>=y and q_y[1]<=abs(x) and len(quarters)>1 or q_x[0]<=y and q_x[1]>=y and q_y[0]>=x and q_y[1]<=x:
                        # third quarter 6.octant
                        points3.add((-y,x))
                        if -y<xy[1][0]:
                            xy[1] = [-y,x]
                        elif -y==xy[1][0]:
                            if x<xy[1][1]:
                                xy[1] = [-y,x]
        if 4 in quarters:
            if not (4 in q):
                # add all points of the quarter completely
                # fourth quarter 7.octant
                points4.add((-y,-x))
                # fourth quarter 8.octant
                points4.add((-x,-y))
            else:
                # start or end point in this quarter?
                if q[0] == 4:
                    # start point
                    if q_x[0]>=y and q_y[0]<=abs(-x) and len(quarters)>1 or q_x[0]>=y and q_x[1]<=y and q_y[0]<=x and q_y[1]>=x:
                        # fourth quarter 7.octant
                        points4.add((-y,-x))
                        if -y<xy[0][0]:
                            xy[0] = [-y,-x]
                        elif -y==xy[0][0]:
                            if -x>xy[0][1]:
                                xy[0] = [-y,-x]
                    if q_x[0]>=x and q_y[0]<=abs(-y) and len(quarters)>1 or q_x[0]>=x and q_x[1]<=x:
                        # fourth quarter 8.octant
                        points4.add((-x,-y))
                        if -x<xy[0][0]:
                            xy[0] = [-x,-y]
                        elif -x==xy[0][0]:
                            if -y>xy[0][1]:
                                xy[0] = [-x,-y]
                if q[1] == 4:
                    # end point
                    if q_x[1]<=y and q_y[1]>=x and len(quarters)>1 or q_x[0]>=y and q_x[1]<=y  and q_y[0]<=x and q_y[1]>=x:
                        # fourth quarter 7.octant
                        points4.add((-y,-x))
                        if -x<xy[1][1]:
                            xy[1] = [-y,-x]
                        elif -x==xy[1][1]:
                            if -y>xy[1][0]:
                                xy[1] = [-y,-x]
                    if q_x[1]<=x and q_y[1]>=abs(-y) and len(quarters)>1 or q_x[0]>=x and q_x[1]<=x:
                        # fourth quarter 8.octant
                        points4.add((-x,-y))
                        if -y<xy[1][1]:
                            xy[1] = [-x,-y]
                        elif -y==xy[1][1]:
                            if -x>xy[1][0]:
                                xy[1] = [-x,-y]
        if switch < 0:
            switch = switch + (4 * x) + 6
        else:
            switch = switch + (4 * (x - y)) + 10
            y = y - 1
        x = x + 1

    if 1 in quarters:
        points1_s = list(points1)
        # points1_s.sort() # if your some reason you need them sorted
        points.extend(points1_s)
    if 2 in quarters:
        points2_s = list(points2)
        # points2_s.sort() # if your some reason you need them sorted
        # points2_s.reverse() # # if your some reason you need in right order
        points.extend(points2_s)
    if 3 in quarters:
        points3_s = list(points3)
        # points3_s.sort()
        # points3_s.reverse()
        points.extend(points3_s)
    if 4 in quarters:
        points4_s = list(points4)
        # points4_s.sort()
        points.extend(points4_s)
    return points, xy

def draw_circle_arc(radius, start, end, i_shift, j_shift,
                    clockwise=True):
    p, xy = circle_arc(radius, start, end, clockwise = clockwise)
    points_for_image = []
    # print(f'Функция draw_circle_arc. i_shift = {i_shift}, j_shift = {j_shift}')
    # print('Точки, полученные из функции растеризации дуги circle_arc:')
    for point in p:
        # print(f'Точка с координатами ({point[0]}, {point[1]},')
        points_for_image.append((i_shift+point[0], j_shift+point[1]))
        # print(f'После сдвига на величину половины картинки координаты этой точки: ({i_shift+point[0]}, {j_shift+point[1]}).')
    return points_for_image  #circle_graph, xy

In [ ]:
def Bresenham_arc(radius, start_angle, end_angle, i_shift, j_shift,
                  direction):
    circlearc_points = draw_circle_arc(radius, start_angle, end_angle, i_shift,
                                            j_shift, direction)
    return circlearc_points

In [ ]:
def Bresenham_line(node1_i, node1_j, node2_i, node2_j):

    line_points = []

    # print(f'В функцию растеризации прямой Брезенхэма пришли узлы 1: ({node1_i},
     #{node1_j}), 2: ({node2_i}, {node2_j}).')

    swap = 0
    if abs(node2_j - node1_j) < abs(node2_i - node1_i):
        swap = 1
        # print(f'Разница между координатами по j {abs(node2_j - node1_j)}
        # меньше разницы координат по i {abs(node2_i - node1_i)}. Поэтому swap.')

    if swap:
        i1 = node1_j
        j1 = node1_i
        i2 = node2_j
        j2 = node2_i
    else:
        i1 = node1_i
        j1 = node1_j
        i2 = node2_i
        j2 = node2_j

    delta_j = abs(j2 - j1)
    delta_i = abs(i2 - i1)

    j_max = max(j1, j2)
    j_min = min(j1, j2)
    if j1 == j_min:
        i = i1
        dir_i = i2 - i1
    else:
        i = i2
        dir_i = i1 - i2

    # print(f'В итоге координаты точки 1: ({i1}, {j1}), координаты точки 2:
    #  ({i2}, {j2}).')
    # print(f'Полученные при этом изменения координат для расчетов: delta_i =
    #  {delta_i}, delta_j = {delta_j}.')
    # print(f'Пределы изменения координаты j по горизонтали: от j_min =
    #  {j_min} до j_max = {j_max}.')

    error = 0
    deltaerr = (delta_i + 1)

    if dir_i > 0:
        dir_i = 1
    if dir_i < 0:
        dir_i = -1
    # print(f'Шаг изменения по вертикальной координате i: dir_i = i2 - i1 = {dir_i}.')

    for j in range (j_min, j_max + 1):
        if swap:
            # print(f'Со swap-ом добавляется точка с координатами ({j}, {i}).')
            line_points.append((j, i))
        else:
            # print(f'Без swap добавляется точка с координатами ({i}, {j}).')
            line_points.append((i, j))

        error = error + deltaerr
        if error >= (delta_j + 1):
            i += dir_i
            error = error - (delta_j + 1)

    return line_points

In [ ]:
# Нахождение угла раствора дуги по координатам ее концов и коэффициенту ее
# кривизны.

def solve_curve(p):

    # Кривизна ребра p не может быть больше pi / 2 = 1.57
    eps = 1e-6
    phi = math.pi/1e3

    result = eps * 100
    while result >= eps:
        f = 2 * p * math.sin(phi/2)
        result = abs(f - phi)
        phi = f

    return phi

In [ ]:
# Функция нахождения вектора во встретившемся узле:

def determine_vector(node, k):

    numbers_dq = deque([0, 1, 2, 3, 4, 5, 6, 7])
    numbers_dq.rotate(-(k+4))
    vector_id = numbers_dq.popleft()
    vector = node.vector(vector_id)

    return vector, vector_id

In [ ]:
class Edge:
    def __init__(self, node_1, node_2, length, curvature, bulge,
                 vector1, vector2):
        # print(f'Создаем экземпляр класса Edge.')
        self.node_1 = node_1
        self.node_2 = node_2
        self.length = length
        self.curvature = curvature
        self.bulge = bulge
        self.vector1 = vector1
        self.vector2 = vector2
        # print('Закончили создание экземпляра Edge.')

In [ ]:
# Функция, собирающая все характеристики ребра и формирующая экземпляр ребра.

def describe_edge(node1, vector1, node2, vector2, edge_length):

    # Проверим, есть ли искривление ребра, и рассчитаем его
    # кривизну:
    delta_i = node1.i - node2.i
    delta_j = node1.j - node2.j
    # print(f'Разница координат равна: delta_i = {delta_i}, delta_j = {delta_j}.')
    distance_between_nodes = max(abs(delta_i), abs(delta_j))
    # print(f'Расстояние между узлами равно distance_between_node = '
    #       f'{distance_between_nodes}.')
    curvature = edge_length / distance_between_nodes
    # print(f'Кривизна ребра равна {curvature}.')
    bulge = 'line'
    if curvature > 1:
        # Узнаем в какую сторону выпукло ребро и определим
        # выпуклость ребра bulge:
        # print(f'Кривизна ребра больше 1 и равна {curvature}.')
        if delta_i != 0 and vector2.coordinates[0] != 0:
            coefficient = delta_i / vector2.coordinates[0]
            comparative_j = vector2.coordinates[1] * coefficient
            # print(f'Посчитаны величины delta_i = {delta_i}, '
            #       f'coefficient = {coefficient} и comparative_j = '
            #       f'{comparative_j}.')
            # Ребро выпукло вправо от прямой, соединяющей узлы:
            if comparative_j > delta_j:
                bulge = 'east'
            # Ребро выпукло влево от прямой, соединяющей узлы:
            elif comparative_j < delta_j:
                bulge = 'west'
            else:
                bulge = 'line'
        elif delta_i == 0:
            if vector2.coordinates[0] < 0:
                bulge = 'east'
            elif vector2.coordinates[0] > 0:
                bulge = 'west'
            else:
                bulge = 'line'
        elif vector2.coordinates[0] == 0:
            if vector2.coordinates[1] > 0:
                bulge = 'east'
            elif vector2.coordinates[1] < 0:
                bulge = 'west'
            else:
                bulge = 'line'
    else:
        bulge = 'line'

    return Edge(node1, node2, edge_length, curvature, bulge,
                vector1, vector2)

In [ ]:
# Функция запуска алгоритма Ли из всех ключевых узлов для описания структуры
# символа: нахождение всех соединяющих ребер и их коэффициентов кривизны и
# занесение их в структуру композитных ребер.
# При этом кортежи в передаваемом списке bending_nodes должны иметь помимо координат
# узлов также их направляющие векторы. Т.е. структура кортежа, соответствующего
# узлу должна быть такой: (i_node, j_node, vector_1, vector_2), т.е. лучше уже
# сюда передавать узлы-изгибы в виде объектов класса node, которые уже будут
# содержать в своих полях направляющие векторы.

def edges_on_the_picture(img, key_nodes: list, bending_nodes: list):

    img = (img / 255).astype('int16')
    size_i, size_j = img.shape
    # В этом массиве будем отмечать только посещения пикселей волнами:
    visited = np.zeros_like(img, dtype='int8')
    # Карта для обозначения всех узлов символа:
    nodes_map = np.zeros_like(img, dtype='int8')
    dq = deque()
    # Обозначим в матрице visited ключевые узлы и узлы перегибов. Чтобы легко
    # различать ключевые узлы и перегибы будем обозначать ключевые узлы номером
    # узла со знаком минус, а узлы перегибов просто номером узла.
    # Также заведем словарь с узлами, чтобы легче было их находить.
    nodes = {}

    # print(f'Переданные в функцию ключевые узлы:')
    total_number_of_nodes = len(key_nodes) + len(bending_nodes)
    n = 0
    for node in key_nodes:
        # Обозначим ключевые узлы в массиве visited и в словаре nodes:
        n += 1
        nodes_map[node.i][node.j] = n
        nodes[n] = node

    last_key_node_number = n
    # print(f'Переданные в функцию узлы-изгибы:')
    for node in bending_nodes:
        n += 1
        nodes_map[node.i][node.j] = n
        nodes[n] = node

    # У нас есть список экземпляров узлов-изгибов.
    # В каждом таком экземпляре хранятся вектора, выходящие из этого узла. Теперь
    # мы можем создавать экземпляры ребер, заносить туда информацию о направляющих
    # векторах и запускать волну построения структуры символа.

    row = [-1, -1, 0, 1, 1, 1, 0, -1]
    col = [0, 1, 1, 1, 0, -1, -1, -1]
    waves = {}
    wave_number = 0
    for node_index, node in nodes.items():
        # Берем из словаря узлов только ключевые узлы, которые в словаре имеют
        # ключами значения id от 1 до last_key_node_number:
        if node_index <= last_key_node_number:
            i = node.i
            j = node.j
            # Для каждого вектора данного ключевого узла сформируем волну и будем
            # ее отслеживать. Поэтому воспользуемся методом класса node, который
            # вернет нам список id векторов данного узла:
            node_vectors_numbers = node.get_vector_numbers()
            for vector_id in node_vectors_numbers:
                # Узлы-перегибы у нас помечены положительными числами, поэтому
                # волны будем помечать отрицательными:
                nodes_dq = deque()  # Стек для записи кортежей пар номер узла -
                                    # ребро до него от предыдущего узла
                wave_number -= 1
                i_next = i + row[vector_id]
                j_next = j + col[vector_id]
                dq.append((i_next, j_next, wave_number, i, j))

                # Здесь сохраняем id предыдущего узла и id предыдущего вектора, а
                # также расстояние от этого узла до текущего узла, и стек с
                # посещенными узлами:
                waves[wave_number] = [node_index, vector_id, 1, nodes_dq,
                                      node_index, vector_id]

    last_wave_number = wave_number
    # В массиве о посещениях пикселей visited выше отметили только узлы-перегибы положительными
    # id. Это нужно для того, чтобы волна при распространении и встрече такого
    # узла понимала, что дошла до узла-перегиба. Ключевые узлы отмечать в массиве
    # посещений вообще не будем. Запись о ключевом узле находится в записи каждой
    # волны, пущенной из этого узла, словаря waves. И ключевые узлы нужны нам
    # только в качестве конечных узлов, которые соединяются формируемыми
    # композитными ребрами.

    # Заведем словарь, в который будем складывать стеки волн, которые уже
    # достигли ключевого узла или встретили волну, вышедшую из этого узла:

    complete_edges = {}

    while dq:
        i, j, wave_number, i_prev, j_prev = dq.popleft()

        # Вначале проверка - существует ли эта волна еще в словаре waves или она
        # уже была встречена другой волной и ее стек был присоединен к стеку той
        # волны. Если эта волна уже была встерчена другой, то ее записи больше нет
        # в словаре, а сам пиксель мог быть добавлен для следующей итерации
        # обработки. Поэтому, если этой волны уже нет в словаре, то просто
        # переходим к следующему в очереди пикселю:

        # print(f'Очередной пиксель имеет волну с номером {wave_number}.')
        if wave_number not in waves:
            # print(f'Волны с номером {wave_number} уже нет в словаре waves.')
            continue

        dist = waves[wave_number][2]
        dist_next = dist + 1

        # В случаях разветвления волны, которые могут возникнуть из-за
        # неидеальности скелетизирования изображения будем в этом месте создавать
        # новую волну из волны, пиксель фронта которой сейчас проверяем. Будем
        # переписывать в эту волну все характеристики проверяемой волны и
        # присваивать ей новый номер, очередной после сохраненного
        # last_wave_number. В свою очередь last_wave_number будем уменьшать (он -
        # отрицательный) на 1 и сохранять для следующего случая разветвления. Но
        # при этом не нужно менять номер самой волны всякий раз, когда находится
        # пиксель возможного хода, потому что такой пиксель может найтись только 1
        # и тогда эта же самая волна должна будет просто распространяться дальше.
        # Поэтому заведем счетчик кол-ва возможных ходов, и всякий раз, когда этот
        # счетчик будет станет больше 1, будем добавлять новую волну:
        move_on = 0

        # print(f'Значение move_on = {move_on}.')
        # Также добавим счетчик количества свободных пикселей среди соседних. Он
        # нам нужен для того, чтобы понимать, встретили ли мы другую волну на
        # композитном ребре, или же на участке с плохой скелетизацией. Во втором
        # случае мы не должны переписывать стек посещенных узлов встреченной волны.
        # А в первом случае надо этот стек переписать и удалить запись встерченной
        # волны из словаря waves:
        move_on2 = 0
        met_wave_id = 0 # В эту переменную будем запоминать номер встреченной
                        # волны, стек которой надо переписать, когда мы поймем,
                        # что встретили ее на композитном ребре.

        for k in range(8):
            # print(f'Очередной пиксель с номером k = {k} из окружения пикселя с'
            #       f' координатами ({i}, {j}).')

            if wave_number not in waves:
                # print(f'Волны с номером {wave_number} уже нет в словаре waves.')
                break

            i_next = i + row[k]
            j_next = j + col[k]
            # print(f'Координаты этого пикселя: по i: {i} + {row[k]}, по j: {j} + {col[k]}.')
            if i_next == i_prev and j_next == j_prev:
                # print(f'Эти координаты совпадают с координатами обратного пути,'
                #       f' поэтому переходим к другому пикселю.')
                continue

            # Проверим возможность сделать шаг в данном направлении (i_next, j_next):
            if cell_validate(img, i_next, j_next):
                # print(f'Шаг в данном направлении сделать можно, move_on = {move_on}.')
                move_on += 1
                move_on2 +=1
                if move_on == 1:
                    new_wave_number = wave_number
                else:
                    last_wave_number -= 1
                    # Вносим в словарь waves запись о новой волне:
                    new_wave_number = last_wave_number

                    waves[new_wave_number] = copy.deepcopy(waves[wave_number])

                if visited[i_next][j_next] == 0 and \
                                                nodes_map[i_next][j_next] == 0:

                    dq.append((i_next, j_next, new_wave_number, i, j))
                    waves[new_wave_number][2] = dist_next
                    visited[i_next][j_next] = new_wave_number
                # Если волна подошла к узлу-изгибу, то формируем ребро edge со
                # всеми его характеристиками, кладем его в стек ребер этой волны.

                elif nodes_map[i_next][j_next] > last_key_node_number:

                    # Вначале этого блока кода все узлы - и ключевые, и изгибы -
                    # заносятся в словарь nodes с ключами, равными их номерам.
                                        node1_id = waves[wave_number][0]
                    node1 = copy.deepcopy(nodes[node1_id])
                    vector1_id = waves[wave_number][1]
                    vector1 = node1.vector(vector1_id)

                    node2_id = nodes_map[i_next][j_next]
                    node2 = copy.deepcopy(nodes[node2_id])
                    vector2, vector2_id = determine_vector(node2, k)

                    distance_between_nodes = max(abs(node1.i-node2.i),
                                                 abs(node1.j-node2.j))

                    if distance_between_nodes != 0:

                        new_edge = describe_edge(node1, vector1, node2, vector2,
                                                 dist_next)
                        # Теперь достаем из данного узла номер второго вектора и
                        # заносим его в начало записи данной волны в словаре waves
                        # вместе с номером самого узла:
                        # Достаем из экземпляра узла список номеров направляющих
                        # векторов:
                        vector_numbers = node2.get_vector_numbers().copy()
                        vector_numbers.remove(vector2_id)
                        next_vector1_id = vector_numbers[0]

                        dq.append((i_next, j_next, new_wave_number, i, j))
                        # В этом случае берем стек вновь созданной волны и
                        # заносим в него новый узел и ребро, которое было
                        # сформировано между эти узлом и предыдущим:
                        nodes_seq = waves[new_wave_number][3]
                        nodes_seq.append((node2, new_edge))
                        waves[new_wave_number][0] = node2_id
                        waves[new_wave_number][1] = next_vector1_id
                        waves[new_wave_number][2] = 0
                        visited[i_next][j_next] = new_wave_number

                    else:
                        complete_edges[new_wave_number] = waves[new_wave_number]
                        del waves[new_wave_number]

                    # Ко второму ключевому узлу волна подойти не сможет, т.к.
                    # она встретит волну, пущенную из того узла. Если волна
                    # встретила другую волну, вышедшую из второго ключевого узла,
                    # связанных композитным ребром, по которому распространяются
                    # эти волны, то мы перекладываем в данную волну стек
                    # встреченной волны, перекладываем запись данной волны в словарь
                    # complete_edges и удаляем эту запись из словаря waves. Также
                    # не ставим данную волну в очередь для дальнейшей обработки.
                    # Запись о встреченной также удаляем из словаря waves:

                elif visited[i_next][j_next] < 0:
                    visitor = visited[i_next][j_next]
                    # print(f'Волна {new_wave_number} из узла {waves[wave_number][4]}'
                    #       f' прошла дистанцию в {dist_next} пикселей и'
                    #       f' наткнулась на пиксель, помеченный волной c номером'
                    #       f' {visitor}.')

                    if visitor not in waves:
                        # print(f'Но оказалось, что волны с номером {visitor} '
                        #       f'уже в словаре waves нет - остались только ее '
                        #       f'следы. Поэтому просто помечаем этот пиксель '
                        #       f'номером данной волны {new_wave_number} и'
                        #       f' переходим к следующему соседнему пикселю.')

                        visited[i_next][j_next] = new_wave_number
                        dq.append((i_next, j_next, new_wave_number, i, j))
                        continue
                    # Длина пути до данного пикселя от предыдущего узла:
                    visitor_dist = waves[visitor][2]
                    # Встречную волну надо удалять из всех списков и стеков.
                    # Если встречная волна прошла уже какие-то узлы-изгибы, то
                    # берем следующий узел из ее стека.
                    # Если же она еще не прошла никакие узлы-изгибы, то следующий
                    # узел для данной волны будет
                    # ключевым узлом, из которого вышла встречная волна:
    # В кортеже пикселя сохраняем номер волны, по которому ее можно найти в
    # словаре волн.
    # В словаре стоит номер предыдущего узла, номер вектора в этом узле, стек
    # посещенных узлов. Если при этом стек пуст, то значит, предыдущий узел - это
    # ключевой узел, из которого вышла волна. Все экземпляры узлов можно найти в
    # словаре nodes. В самом экземпляре узла можно найти вектор по его id.

                    visitor_previous_node_id = waves[visitor][0]

                    # Проверим, не вышла ли встретившаяся волна из того же самого
                    # ключевого узла. Если она - из того же самого ключевого
                    # узла, тогда просто игнорируем ее. Но тут может быть ошибка,
                    # если в процессе обнаружения ключевых узлов в ключевые был
                    # внесен узел, который имеет петлю без других узлов. Такое
                    # может произойти, например из-за некачественной скелетизации.
                    # Может быть, в этом случае лучше не ставить данную волну
                    # дальше в очередь на обработку.
                    # В этом случае просто удалим данную волну:

                    if visitor_previous_node_id == waves[wave_number][0]:
                        # print(f'У этой волны с номером {new_wave_number}'
                        #       f' предыдущий узел: {waves[wave_number][0]}, '
                        #       f'который совпадает с предыдущим узлом '
                        #       f'{visitor_previous_node_id} встреченной волны '
                        #       f'{visitor}. Поэтому волна с номером '
                        #       f'{new_wave_number} удаляется из словаря waves.')
                        # print(f'В словаре waves к этому моменту содерджатся волны'
                        #       f' c номерами: {list(waves.keys())}.')

                        if move_on > 1:
                            del waves[new_wave_number]
                        else:
                            move_on = 0

                        continue

                    met_wave_id = visitor

                # Для случая достижения волной ключевого узла. При этом, если
                # этот ключевой узел к тому же является еще и предыдущим узлом
                # данной волны, т.е. волна вышла из данного ключевого узла, то мы
                # просто удаляем запись для данной волны из словаря waves. Если
                # же предыдущий узел волны не является данным ключевым узлом, то
                # мы просто формируем новое ребро и добавляем это ребро вместе с
                # данным ключевым узлом в стек посещенных данной волной ребер. В
                # любом случае при достижении ключевого узла волна больше не
                # ставится в очередь для дальнейшей обработки:

                elif nodes_map[i_next][j_next] > 0 \
                          and nodes_map[i_next][j_next] <= last_key_node_number:
                    # print(f'Волна с номером {wave_number} пришла в ключевой узел'
                    #       f' с номером {visited[i_next][j_next]}.')
                    previous_node_id = waves[new_wave_number][0]

                    if previous_node_id != nodes_map[i_next][j_next]:

                        node1 = copy.deepcopy(nodes[previous_node_id])
                        vector1_id = waves[wave_number][1]
                        vector1 = node1.vector(vector1_id)
                        node2_id = nodes_map[i_next][j_next]
                        node2 = copy.deepcopy(nodes[node2_id])
                        vector2, vector2_id = determine_vector(node2, k)
                        distance_between_nodes = max(abs(node1.i-node2.i),
                                                 abs(node1.j-node2.j))

                        if distance_between_nodes != 0:

                            new_edge = describe_edge(node1, vector1, node2,
                                                     vector2, dist_next)
                            nodes_seq = waves[new_wave_number][3]
                            nodes_seq.append((node2, new_edge))

                            complete_edges[new_wave_number] = waves[new_wave_number]
                            del waves[new_wave_number]
                    else:
                        del waves[wave_number]

        if move_on2 == 1 and met_wave_id:
            node2_id = waves[met_wave_id][0]
            node2 = copy.deepcopy(nodes[node2_id])
            vector2_id = waves[met_wave_id][1]
            vector2 = node2.vector(vector2_id)
            node1_id = waves[wave_number][0] # Берем id предыдущего узла
            node1 = copy.deepcopy(nodes[node1_id])

            vector1_id = waves[wave_number][1]
            vector1 = node1.vector(vector1_id)
            edge_length = dist_next + visitor_dist

            distance_between_nodes = max(abs(node1.i-node2.i),
                                          abs(node1.j-node2.j))
            edge_to_add = 0
            if distance_between_nodes != 0:

                edge_to_add = describe_edge(node1, vector1, node2,
                                            vector2, edge_length)

            # Для записи узлов и ребер встречной волны берем стек новой
            # волны, если произошло разветвление, либо берем стек этой
            # волны, если разветвления не произошло:
            nodes_seq = waves[new_wave_number][3]

            # Стек с занесенными ребрами встречной волны:
            visitor_nodes = waves[met_wave_id][3]

            visitor_nodes = list(visitor_nodes)

            while visitor_nodes:

                node_to_add, next_edge = visitor_nodes.pop()
                if edge_to_add:
                    nodes_seq.append((node_to_add, edge_to_add))
                edge_to_add = next_edge

            #  Добавление последнего, ключевого, узла встреченной волны:
            node_to_add_id = waves[met_wave_id][4]
            node_to_add = copy.deepcopy(nodes[node_to_add_id])
            if edge_to_add:
                nodes_seq.append((node_to_add, edge_to_add))

            # Обновим запись данной волны в словаре waves:
            waves[new_wave_number][0] = node_to_add_id
            waves[new_wave_number][1] = waves[met_wave_id][5]
            waves[new_wave_number][2] = 0

            complete_edges[new_wave_number] = waves[new_wave_number]
            del waves[new_wave_number]

            # Удаляем встретившуюся волну из словаря, а эту волну
            # оставляем в словаре, просто не ставим в дальнейшую обработку
            # ее пиксель:
            del waves[met_wave_id]

    # В конечном итоге мы получили словарь complete_edges, в котором остались записи
    # для тех волн, которые софрмировали полные композитные ребра от одного
    # ключевого узла до другого.

    # Необходимо еще обработать вариант с разветвлением волн в самом ключевом
    # узле, как это происходит в одном из символов из набора данных.

    waves.update(complete_edges)

    edges_list = []
    for wave in waves.values():
        edges_list.extend(wave[3])

    return edges_list

In [ ]:
def node_guide_vectors_calc(img, nodes_dict, other_nodes_dict):

    """Это функция расчета направляющих векторов для узлов, переданных в словаре
    nodes_dict. Перед вызовом этой функции формируются 2 словаря: один для
    ключевых узлов, другой - для узлов-изгибов. Ключами в словарях служат id
    узлов. Эти словари нужно сформировать в одном месте, чтобы последовательно
    сформировать для узлов их id без нарушения последовательности. Затем словарь
    с ключевыми узлами передается в эту функцию для формирования векторов.

    Словарь с узлами-изгибами будет передаваться из функции проверки узлов из
    словаря nodes_to_check после каждой итерации. Затем эта функция возвращает
    в функцию проверки узлов словарь с векторами, функция проверки проверяет
    углы между направляющими векторами, определяет, какой из узлов является
    ключевым, а какой - изгибом. Заново готовит словарь для проверки и отправляет
    сюда, в эту функцию.
    """

    MAX_DISTANCE = 10  # Константа, которая задает количество шагов, достаточных
                        # для расчета направляющего вектора.

    img = (img / 255).astype('int16')
    visited = np.zeros_like(img)

    # Заполним массив visited всеми имеющимися в символе узлами - и теми, для
    # которых надо посчитать вектора, и теми, которые также есть в символе. Все
    # узлы важны, т.к. они меняют направление ребра, и, по сути, направляющий
    # вектор может быть рассчитан только до какого-то узла.
    def fill_visited(nodes_to_put):
        for node_id, coordinates in nodes_to_put.items():
            i, j = coordinates
            visited[i][j] = node_id

    fill_visited(nodes_dict)
    fill_visited(other_nodes_dict)

    dq = deque()
    row = [-1, -1, 0, 1, 1, 1, 0, -1]
    col = [0, 1, 1, 1, 0, -1, -1, -1]

    # Для каждого ключевого узла, которые нужно проверить, сделаем по одному
    # шагу в каждом возможном направлении, сформируем тем самым вектора для
    # каждого из этих направлений и внесем эти пиксели в очередь для дальнейшей
    # обработки:
    node_vectors = {}
    for node_id, coordinates in nodes_dict.items():

        # Считаем координаты очередного проверяемого узла:
        i_node, j_node = coordinates
        node_vectors[node_id] = {}

        # Инициализируем волны, расходящиеся от данного узла по разным
        # ребрам.
        dist = 1

        for k in range(8):

            i_next = i_node + row[k]
            j_next = j_node + col[k]

            if cell_validate(img, i_next, j_next):
                # Поскольку обход окружающих пикселей происходит всегда по одной и той же
                # схеме, то можно к этим направлениям P2 - P9 привязать и вектора. И
                # привязку эту делать через значения k. Это значение k будем заносить
                # в атрибут vector_id экземпляра guiding_vector.
                vector_id = k
                vector = guiding_vector((i_node, j_node), node_id, vector_id, dist)
                vector.update(i_next, j_next)

                # Для того, чтобы не потерять информацию о направлении, откуда
                # пришла данная волна, будем сохранять предыдущий пиксель пути
                # этой волны в двух последних элементах кортежа информации о
                # пикселе:
                dq.append((i_next, j_next, dist, vector, i_node, j_node))

    # Теперь запускаем волновой алгоритм и считаем каждый направляющий вектор:
    while dq:
        i, j, dist, vc, i_prev, j_prev = dq.popleft()
        dist_next = dist + 1

        move_on = 0
        for k in range(8):

            if dist_next > MAX_DISTANCE:
                break

            i_next = i + row[k]
            j_next = j + col[k]

            # Проверка на обратный путь, который нам не нужен:
            if i_next == i_prev and j_next == j_prev:
                continue

            # При расчете направляющих векторов обязательно нужно делать проверку
            # на встречу и ключевых узлов, и узлов изгибов! Ключевые вектора,
            # очевидно, представляют конец ребра. А если не учитывать узлы
            # изгибов, то может возникнуть такая ситуация, когда из узла выходит
            # петля, и направляющий вектор будет показывать внутрь этой петли.
            # По сути изгибы меняют направление ребра.

            if cell_validate(img, i_next, j_next) and \
                                              visited[i_next][j_next] == 0:
                move_on += 1

                # Проверим, произошло ли уже разветвление этой волны. Если
                # разветвление уже было, то атрибут distance экземпляра
                # guiding_vector в данной волне уже был изменен другим пикселем
                # на значение, равное как раз dist_next. И если разветвление уже
                # было, то поставим флажок vc.branching в 1 и не будем добавлять
                # настоящий пиксель в очередь. Также необходимо проверить сам
                # этот флажок, который мог быть изменен другим пикселем на
                # предыдущей итерации уже после того, как данный пиксель произвел
                # изменения.
                if vc.branching == 0 and vc.distance == dist_next:
                    vc.branching = 1
                    vc.rollback()
                    vc.distance = dist

                    # Если одна из разветвленных волн уже была обработана и
                    # изменила vc.distance, то, понятно, что произошло разветвление
                    # и та волна пошла на очередной круг обработки, поэтому она
                    # ничего не вписала в словарь с найденными векторами. Поэтому
                    # данная волна, которая подошла к этой же дистанции теперь,
                    # может вписать данный вектор в словарь векторов:
                    node_vectors[vc.node_id][vc.vector_id] = vc

                elif vc.branching == 0 and vc.distance != dist_next:
                    vc.distance = dist_next
                    vc.update(i_next, j_next)
                    dq.append((i_next, j_next, dist_next, vc, i, j))

        if move_on == 0 and (vc.vector_id not in node_vectors[vc.node_id]):

            # Это условие нужно для первой из волн разветвления, которая
            # была поставлена в очередь на предыдущей итерации и которая на
            # этой итерации уже не может продвинуться дальше. Запись вектора
            # в словарь нужна также для случая распространения одной волны
            # без разветвлений. Поэтому записываем посчитанный вектор в словарь
            # и заносим его также в словарь векторов узла с номером node_id:
            node_vectors[vc.node_id][vc.vector_id] = vc

    return node_vectors

In [ ]:
def find_sample_bounds (img):
    sizeI, sizeJ = img.shape

    i_min, j_min, i_max, j_max = 0, 0, sizeI - 1, sizeJ - 1

    # Найдем верхнюю границу символа на изображении:
    pixel_found = 0
    i = 0
    while not pixel_found and i < sizeI:
        j = 0
        while not pixel_found and j < sizeJ:
            if img[i, j] == 1:
                pixel_found = 1
                i_min = i
            j += 1
        i += 1

    # Найдем левую границу символа на изображении:
    pixel_found = 0
    j = 0
    while not pixel_found and j < sizeJ:
        i = 0
        while not pixel_found and i < sizeI:
            if img[i, j] == 1:
                pixel_found = 1
                j_min = j
            i += 1
        j += 1

    # Найдем нижнюю границу символа на изображении:
    pixel_found = 0
    i = sizeI - 1
    while not pixel_found and i >= 0:
        j = 0
        while not pixel_found and j < sizeJ:
            if img[i, j] == 1:
                pixel_found = 1
                i_max = i
            j += 1
        i -= 1

    # Найдем правую границу символа на изображении:
    pixel_found = 0
    j = sizeJ - 1
    while not pixel_found and j >= 0:
        i = 0
        while not pixel_found and i < sizeI:
            if img[i, j] == 1:
                pixel_found = 1
                j_max = j
            i += 1
        j -= 1

    return i_min, j_min, i_max, j_max

In [ ]:
class Node:

    def __init__(self, node_i, node_j, node_id):
        self.i = node_i
        self.j = node_j
        self.id = node_id
        self._vc = {}

    def new_vector(self, vector_to_add):
        vector_id = vector_to_add.vector_id
        self._vc[vector_id] = vector_to_add

    def vector(self, vector_id):
        return self._vc[vector_id]

    def get_vector_numbers(self):
        return list(self._vc.keys())

In [ ]:
# Функция генерации множества прямых: горизонтальных, вертикальных, наклонных с
# положительной и отрцательной производными.

def secant_generator(size_I, size_J):
    # Генерация прямых под наклоном с большим углом к вертикали:
    LINES_NUM = 20  # Количество прямых, пересекающих вертикальную границу.
    LINES_IN_PIC = 10

    secants = {}
    delta_b = round(size_I / LINES_IN_PIC)

    # Сгенерируем горизонтальные линии и внесем их в список secants:
    start_j = 0
    end_j = size_J - 1
    secants[0] = []
    for n in range(1, LINES_IN_PIC):
        i = size_I - n * delta_b
        secants[0].append(((i, start_j), (i, end_j)))

    # Сгенерируем вертикальные линии и внесем их в список secants:
    start_i = 0
    end_i = size_I - 1
    sep = round(size_J / LINES_IN_PIC)
    secants[90] = []
    for n in range(1, LINES_IN_PIC):
        j = n * sep
        secants[90].append(((start_i, j), (end_i, j)))

    # Сгенерируем наклонные линии с положительной производной:
    deg_list = [15, 30, 45, 60]
    k_list = [math.tan(math.radians(degrees)) for degrees in deg_list]
    for index, k in enumerate(k_list):    # j_low < size_J:
        inclination = deg_list[index]
        secants[inclination] = []
        i_b = 0
        while i_b <= LINES_NUM:
            i_b += 1
            line_out = 0
            i_up = 0
            j_up = round((i_up - (size_I - delta_b * i_b)) / k)
            if j_up < 0:
                j_up = 0
                i_up = size_I - delta_b * i_b
            elif j_up > (size_J - 1):
                line_out = 1
            i_low = size_I - 1
            j_low = round((i_low - (size_I - delta_b * i_b)) / k)
            if j_low > (size_J - 1):
                j_low = size_J - 1
                i_low = round(j_low * k + (size_I - delta_b * i_b))
                if i_low < 0:
                    line_out = 1
            if line_out == 0:
                secants[inclination].append(((i_up, j_up), (i_low, j_low)))
            # j_up = - round((size_I - delta_b * i_b) / k)

    # Сгенерируем наклонные линии с положительной производной:
    deg_list = [ - 15, - 30, - 45, - 60]
    k_list = [math.tan(math.radians(degrees)) for degrees in deg_list]
    for index, k in enumerate(k_list):    # j_low < size_J:
        inclination = deg_list[index]
        secants[inclination] = []
        i_b = 0
        while i_b <= LINES_NUM:
            i_b += 1
            line_out = 0
            i_up = 0
            j_up = round((i_up - delta_b * i_b) / k)
            if j_up > (size_J - 1):
                j_up = size_J - 1
                i_up = round(j_up * k + delta_b * i_b)
                if i_up > size_I - 1:
                    line_out = 1
            i_low = size_I - 1
            j_low = round((i_low - delta_b * i_b) / k)
            if j_low < 0:
                j_low = 0
                i_low = round(j_low * k + delta_b * i_b)
            if j_low > (size_J - 1):
                line_out = 1
            if line_out == 0:
                secants[inclination].append(((i_up, j_up), (i_low, j_low)))
            # j_low = round((i_low - (size_I - delta_b * i_b)) / k)

    line_points = {}
    for inclination_deg, lines in secants.items():
        line_points[inclination_deg] = []
        for line in lines:
            start, end = line
            i1, j1 = start
            i2, j2 = end
            line_points[inclination_deg].append(Bresenham_line(i1, j1, i2, j2))

    return line_points

In [ ]:
def model_struct(edges_set, sizeI, sizeJ):

    # Заведем список, в который будем собирать все рассчитанные точки для всех
    # ребер:
    points_of_edges = []

    for _, edge in edges_set:

        # Выбираем, какой из двух узлов лежит выше. Узлом 1 будет узел, который
        # лежит выше.
        if edge.node_1.i <= edge.node_2.i:
            i1 = edge.node_1.i
            j1 = edge.node_1.j
            j2 = edge.node_2.j
            i2 = edge.node_2.i
        else:
            i1 = edge.node_2.i
            j1 = edge.node_2.j
            j2 = edge.node_1.j
            i2 = edge.node_1.i

        curvature = edge.curvature
        distance = np.sqrt(np.power((i1 - i2), 2) + pow((j1 - j2), 2))

        curve_length = distance * curvature
        bulge = edge.bulge

        # Поставил пока ограничения на максимальное значение curvature. Потом
        # надо его снять, но вначале надо проверять опять как волны гуляют по
        # символу.
        if curvature > 1.1 and curvature < 2 and bulge != 'line':

            angle = solve_curve(curvature)
            radius = curve_length / angle

            if j1 != j2:
                alpha = math.atan(abs(i1 - i2) / abs(j1 - j2)) # Угол наклона прямой,
                                      # соединяющей два узла относительно оси j.
            else:
                alpha = 0

            j_max = max(edge.node_1.j, edge.node_2.j)
            if j1 == j_max:
                if bulge == 'west': # Выпуклость ребра влево или вниз.
                    beta = angle / 2 + (math.pi / 2 - alpha)
                    i_c = i1 + radius * math.sin(beta)
                    j_c = j1 + radius * math.cos(beta)

                    # Начальный угол, который правильный для движения против
                    # часовой стрелки.
                    end_angle = (math.atan((i_c - i1 + 1) / (j_c - j1 + 1)) \
                                                  - math.pi / 2) + 2 * math.pi
                    start_angle = end_angle - angle
                    clockwise = True
                elif bulge == 'east':
                    beta = alpha - angle / 2
                    i_c = i2 - radius * math.cos(beta)
                    j_c = j2 - radius * math.sin(beta)
                    start_angle = math.pi / 2 - \
                                      math.atan((i_c - i1 + 1) / (j1 - j_c + 1))
                    end_angle = start_angle + angle
                    clockwise = True
            else:
                if bulge == 'west':
                    beta = alpha - angle / 2
                    i_c = i2 - radius * math.cos(beta)
                    j_c = j2 + radius * math.sin(beta)
                    start_angle = (math.pi / 2 - \
                                   math.atan((i2 - i_c + 1) / (j_c - j2 + 1))) \
                                                                      + math.pi
                    end_angle = start_angle + angle
                    clockwise = True
                elif bulge == 'east':
                    beta = angle / 2 + (math.pi / 2 - alpha)
                    i_c = i1 + radius * math.sin(beta)
                    j_c = j1 - radius * math.cos(beta)
                    start_angle = math.pi / 2 - \
                                      math.atan((i_c - i1 + 1) / (j1 - j_c + 1))
                    end_angle = start_angle + angle
                    clockwise = True

            # !!!! Функция Bresenham_arc отсчитывает угол от вертикали 90 градусов, т.е.
            # обычные 90 градусов для нее - 0 градусов. Увеличение угла идет по
            # часовой стрелке. Если направление обхода выбрано по часовой стрелке,
            # то будет нарисована дуга в направлении увеличения угла. Если выбрано
            # направление обхода против часовой стрелки, то будет нарисована
            # противолежащая дуга, т.е. от начального угла до конечного, но в
            # противоположном направлении.
            # !!!! И отрицательные углы она вообще не воспринимает - просто
            # игнорирует их.

            new_edge_points = Bresenham_arc(radius, start_angle,
                                                 end_angle, i_c, j_c, clockwise)
            rounded_coordinates = [(round(i), round(j)) for i, j in
                                   new_edge_points if round(i) > 0 and
                                   round(i) < sizeI and round(j) > 0 and
                                   round(j) < sizeJ]
            points_of_edges.extend(rounded_coordinates)

            for coord in rounded_coordinates:
                i, j = coord

        else:
            new_edge_points = Bresenham_line(i1, j1, i2, j2)
            rounded_coordinates = [(round(i), round(j)) for i, j in
                                   new_edge_points if round(i) > 0 and
                                   round(i) < sizeI and round(j) > 0 and
                                   round(j) < sizeJ]
            points_of_edges.extend(rounded_coordinates)

            for coord in new_edge_points:
                i, j = coord
                # print(f'Координаты очередной точки прямой: i = {i}, j = {j}.')

    return points_of_edges

In [ ]:
def resize_struct_pic(new_size_i, new_size_j, i_min, j_min, i_max, j_max, edges_list):

    resized_edges_list = copy.deepcopy(edges_list)

    old_length_i = i_max - i_min + 1
    old_length_j = j_max - j_min + 1

    if old_length_i != 0 and old_length_j != 0:
        resize_coeff_i = int(new_size_i / old_length_i)
        resize_coeff_j = int(new_size_j / old_length_j)
        resize_coeff = min(resize_coeff_i, resize_coeff_j)

    zero_i = old_length_i / 2 + i_min - 1 / 2
    zero_j = old_length_j / 2 + j_min - 1 / 2

    for _, stroke in resized_edges_list:
        new_edge_i1 = round((stroke.node_1.i - zero_i) * resize_coeff + new_size_i / 2)
        new_edge_j1 = round((stroke.node_1.j - zero_j) * resize_coeff + new_size_j / 2)
        new_edge_i2 = round((stroke.node_2.i - zero_i) * resize_coeff + new_size_i / 2)
        new_edge_j2 = round((stroke.node_2.j - zero_j) * resize_coeff + new_size_j / 2)

        stroke.node_1.i = new_edge_i1
        stroke.node_1.j = new_edge_j1
        stroke.node_2.i = new_edge_i2
        stroke.node_2.j = new_edge_j2

    return resized_edges_list

In [ ]:
# Функция поиска и подсчета количества пересечений выбранных прямых и символа.
# Эта функция также возвращает список с координатами этих пересечений для
# построения картинки.
def find_lines_crosses(img, probe_lines):

    sizeI, sizeJ = img.shape

    cross_coordinates = []

    row = [-1, -1, 0, 1, 1, 1, 0, -1]
    col = [0, 1, 1, 1, 0, -1, -1, -1]
    sample_features = []
    for lines in probe_lines.values():
        for line in lines:
            new_feature = 0
            visited = np.zeros((sizeI, sizeJ))
            for point in line:
                i, j = point
                one_cross = 0
                if img[i, j] == 255:
                    visited[i, j] = 1
                    # Проверим рядом стоящие пиксели, относятся ли они к
                    # одному пересечению символа:
                    for k in range(8):
                        i_near = i + row[k]
                        j_near = j + col[k]
                        if i_near >= 0 and i_near < sizeI and \
                                          j_near >= 0 and j_near < sizeJ:
                            if visited[i_near, j_near] == 1:
                                one_cross = 1
                    if not one_cross:
                        new_feature += 1
                        cross_coordinates.append((i, j))
            sample_features.append(new_feature)

    # print(f'Для этого изображения собран вектор признаков sample_features {sample_features}.')
    return sample_features, cross_coordinates

In [ ]:
# Функция отмечает на изображение символа крайние точки и найденный ноль, т.е.
# центр изображения символа:
def sample_bounds_pic(img, i_min, j_min, i_max, j_max):
    crop_center_pic = pic_colored_copy(img)

    # Обозначим на изображении символа крайние углы изображения
    # print('Обозначение крайних точек на изображении:')
    for i, j in [(i_min, j_min), (i_max, j_max)]:
        # print(f'i = {i}, j = {j}.')
        crop_center_pic[i, j, 0] = 255
        crop_center_pic[i, j, 1] = 0
        crop_center_pic[i, j, 2] = 0

    old_length_i = i_max - i_min
    old_length_j = j_max - j_min
    zero_i = old_length_i // 2 + old_length_i % 2 + i_min
    zero_j = old_length_j // 2 + old_length_j % 2 + j_min

    # print(f'Найденный центр изображения: zero_i = {zero_i}, zero_j = {zero_i}.')
    crop_center_pic[zero_i, zero_j, 0] = 0
    crop_center_pic[zero_i, zero_j, 1] = 255
    crop_center_pic[zero_i, zero_j, 2] = 0

    return crop_center_pic

In [ ]:
# Функция изображает на скелетизированном очищенном изображении символа все
# ребра, которые были найдены:
def skeleton_edges_pic(img, edges_list):
    edges_picture = pic_colored_copy(img)

    count = 0
    for _, edge in edges_list:
        i1 = edge.node_1.i
        j1 = edge.node_1.j
        i2 = edge.node_2.i
        j2 = edge.node_2.j
        if edges_picture[i1][j1][0] == 255 and \
            edges_picture[i1][j1][1] == 255 and \
            edges_picture[i1][j1][2] == 255:
            edges_picture[i1][j1][0] = 0
            edges_picture[i1][j1][1] = 0
            edges_picture[i1][j1][2] = 0
        edges_picture[i1][j1][count % 3] = 255
        if edges_picture[i2][j2][0] == 255 and \
            edges_picture[i2][j2][1] == 255 and \
            edges_picture[i2][j2][2] == 255:
            edges_picture[i2][j2][0] = 0
            edges_picture[i2][j2][1] = 0
            edges_picture[i2][j2][2] = 0
        edges_picture[i2][j2][count % 3] = 255
        count += 1

    return edges_picture

In [ ]:
"Функция обозначает на очищенной скелетизированной картинке ключевые узлы и изгибы."
def sample_nodes_pic(img, key_nodes, bending_nodes):
    colored_cleaned_picture = pic_colored_copy(img)

    for node in key_nodes:
        i = node.i
        j = node.j
        colored_cleaned_picture[i, j, 0] = 255
        colored_cleaned_picture[i, j, 1] = 0
        colored_cleaned_picture[i, j, 2] = 0
    for node in bending_nodes:
        i = node.i
        j = node.j
        if colored_cleaned_picture[i, j, 0] == 255 and \
            colored_cleaned_picture[i, j, 1] == 0 and \
            colored_cleaned_picture[i, j, 2] == 0:
            colored_cleaned_picture[i, j, 2] = 255
        else:
            colored_cleaned_picture[i, j, 0] = 0
            colored_cleaned_picture[i, j, 1] = 0
            colored_cleaned_picture[i, j, 2] = 255

    return colored_cleaned_picture

In [ ]:
# Функция отмечает найденные источники возбуждения волн на скелетизированном
# изображении.
def wave_sources_pic(img, sources):
    marked_wave_skeleton = pic_colored_copy(img)

    # Отобразим источники волн на селетизированном изображении:
    for wave_source in sources:
        i, j = wave_source
        marked_wave_skeleton[i][j][0] = 255
        marked_wave_skeleton[i][j][1] = 0
        marked_wave_skeleton[i][j][2] = 0

    return marked_wave_skeleton

In [ ]:
def pic_colored_copy(bw_img):
    sizeI, sizeJ = bw_img.shape
    colored_copy = np.zeros((sizeI, sizeJ, 3))
    for i in range(3):
        colored_copy[:, :, i] = bw_img.copy()

    return colored_copy

In [ ]:
# Функция отмечает на скелетизированном изображении ключевые узлы и спорные узлы,
# которые соответствуют шаблонам и которые нужно проверить с помощью направляющих
# векторов:

def key_bending_nodes_pic(img, key_nodes, nodes_to_check):
    marked_skeleton = pic_colored_copy(img)

    for node in key_nodes:
        i = node.i
        j = node.j
        marked_skeleton[i][j][0] = 255
        marked_skeleton[i][j][1] = 0
        marked_skeleton[i][j][2] = 0
    for node in nodes_to_check:
        i, j = node
        if marked_skeleton[i][j][0] == 255 and \
          marked_skeleton[i][j][1] == 0 and \
          marked_skeleton[i][j][2] == 0:

            marked_skeleton[i][j][2] = 255
        else:

            marked_skeleton[i][j][0] = 0
            marked_skeleton[i][j][1] = 0
            marked_skeleton[i][j][2] = 255

    return marked_skeleton

In [ ]:
# Функция отмечает на скелетизированном изображении все найденные на этом изображении
# символы:
def all_samples_pic(img, samples, node_sequences):
    samplePicture = pic_colored_copy(img)

    samplesNumber = len(samples)
    for sample in range(samplesNumber):
        # print(f'Символ с номером {sample}')
        waves = samples[sample]

        for wave in waves:
            nodes = node_sequences[wave]

            for node in nodes:
                i, j = node
                # print(f'{(i, j)}', end=', ')
                samplePicture[i, j, 0] = 0
                samplePicture[i, j, 1] = 0
                samplePicture[i, j, 2] = 0
                if sample % 3 == 0:
                    samplePicture[i, j, 0] = 255
                elif sample % 3 == 1:
                    samplePicture[i, j, 1] = 255
                elif sample % 3 == 2:
                    samplePicture[i, j, 2] = 255

    return samplePicture

In [ ]:
# Функция построения участков изображения с главными диагоналями для сбора
# признаков в виде среднего количества пикселей на главных дагоналях в этом
# участке.

def diagonal_features(img):

    # Размеры участка разбиения изображения:
    PIECE_I = 4
    PIECE_J = 8
    # img = (img / 255).astype('int16')
    sample_features = []

    for i_start in range(5, 85, 4):
        for j_start in range(2, 58, 8):

            # Заведем словарь с диагоналями для подсчет количества пикселей изображения
            # структурной модели символа на каждой из них. В качестве ключей словаря
            # будут выступать кортежи с начальными координатами диагоналей, а значениями
            # будут количество пикселей, лежащих на каждой из них. Начальными
            # координатами диагоналей, упирающихся в левую граниу участка разбиения
            # будут кортежу вида (i, 0) и (0, j) для диагоналей, упирающихся в верхнюю
            # границу участка разбиения:
            diagonals = {}
            for i in range(PIECE_I):
                diagonals[(i, 0)] = 0
            for j in range(1, PIECE_J):
                diagonals[(0, j)] = 0

            i_end = i_start + PIECE_I
            j_end = j_start + PIECE_J
            for i in range(i_start, i_end):
                for j in range(j_start, j_end):
                    if img[i, j] == 255:
                        if (i - i_start) < (j - j_start):
                            diagonals[(0, j - j_start)] +=1
                        else:
                            diagonals[(i - i_start, 0)] +=1
            average_pixels_num = 0
            for diagonal_pixels in diagonals.values():
                average_pixels_num += diagonal_pixels
            average_pixels_num += round(diagonal_pixels / len(diagonals))
            sample_features.append(average_pixels_num)

    return sample_features

In [ ]:
# Функция обработки символов и сбора их признаков.

import sys  # Импортируем модуль для останова программы

def collect_letters_features(letters_path, letter, probe_lines, NEW_SIZE_I,
                             NEW_SIZE_J):

    # letter_instances_path = os.path.join(letters_path, letter)
    file_names = os.listdir(letters_path)
    number_of_files = len(file_names)
    # print(f'В функцию collect_letters_features передан адрес папки {letters_path}'
    #       f', и буква {letter}.)')
    # print(f'В папке находятся изображения буквы с именами: {file_names}.')

    # limit = len(file_names)
    picture_num = 0
    letter_features = pd.DataFrame()
    # for i in range(5, 7):
    for file in file_names:

        # file = file_names[i]
        picture_num += 1
        print(f'Изображения буквы {letter}. Картинка номер {picture_num} из файла {file}.')

        img_path = os.path.join(letters_path, file)
        # print(f'Читаем файл {img_path}.')
        original_img = cv2.imread(img_path)
        original_img = cv2.cvtColor(original_img, cv2.COLOR_BGR2RGB)

        bw_img, binary = prepare_pictures(img_path)

        # Вот этот блок в случае датасета из CoMNIST закомментируем:
###################################
        histI, histJ = histogram(binary)
        bin_i = len(histI)
        rows = [i for i in range(bin_i)]
        bin_j = len(histJ)
        columns = [i for i in range(bin_j)]

        thinned_binary = thin_picture(binary, histI, histJ)
####################################

        # В случае датасета из CoMNIST данная строка не нужна:
        BW_Original = thinned_binary.copy()
        # BW_Original = binary

        # Инвертируем полученные бинаризированные изображения:
        # В случае с датасетом CoMNIST инвертирование не требуется, потому что в
        # датасете буквы написаны белым на черном.
        BW_Original = cv2.bitwise_not(BW_Original)

        # Перед применением алгоритмов скелетизации Зонга-Суня и Ву-Цая проведем
        # предварительную обработку изображения по методу, предложенному в работах
        # Хаустова:
        # BW_Original = preprocess_char(BW_Original)

        "Применим алгоритм Зонга-Суня для начальной скелетизации символа:"
        BW_SkeletonZS = zhangSuen(BW_Original)
        # BW_Skeleton = BW_Original
        "Далее применим алгоритм Ву-Цая для окончательной скелетизации символа:"
        Wu_Tsai(BW_SkeletonZS)

        BW_SkeletonWT = Wu_Tsai(BW_SkeletonZS)

        # Найдем точки для старта алгоритма Ли. Заведем также список объектов,
        # которые нужно будет удалить с изображения как артефакты. Этот список
        # для начала передадим в функцию get_wave_sources, чтобы она внесла в этот
        # список одиноко стоящие пиксели, которые с малой долей вероятности могут
        # относиться к символу:
        samples_to_delete = []
        sources, samples_to_delete = get_wave_sources(BW_SkeletonWT, samples_to_delete)

        # Теперь запустим первый обход изображения алгоритмом Ли из найденных
        # стартовых пикселей. Сформируем списки ключевых пикселей и отдельных фигур,
        # изображенных на поданной на вход картинке:
        key_nodes, nodes_to_check, samples, node_sequences = get_key_nodes(BW_SkeletonWT, sources)

        # Найдем в списке всех найденных на картинке символов samples символы, у
        # которых в структуре не больше 2-х узлов. С большой вероятностью эти символы
        # являются артефактами, осавшимися на картинке после ее чистки, и их можно
        # без потери информации удалить.
        limit = len(samples)
        deleted_samples_indexes = []
        for i in range(limit):
            sample = samples[i]
            nodes_in_sample = 0
            for wave in sample:
                nodes_in_sample += len(node_sequences[wave])

            # Если количество узлов всех типов в символе 1 или 2, то с большой
            # вероятностью - это не часть символа, а какой-то артефакт (за исключением
            # таких букв как ё и й). Поэтомму выбираем из всех волн в символе волну с
            # номером 0 и из нее выбираем узел с номером 0 и ставим этот узел в список
            # на удаление. Потом этот список будет передан в волновой алгоритм для
            # удаления этих артефактов:
            if nodes_in_sample < 3:
                samples_to_delete.append(list(node_sequences[list(sample)[0]])[0])

                # Включаем индекс этого символа в список на удаление из списка
                # samples, а также удаляем все волны, которые включает в себя этот
                # символ, из словаря node_sequences. Также удаляем все узлы этого
                # символа из списка узлов, которые необходимо проверить,
                # nodes_to_check и из списка ключевых узлов key_nodes:
                deleted_samples_indexes.append(i)
                for wave in sample:
                    for node in node_sequences[wave]:
                        nodes_to_check.discard(node)
                        key_nodes.discard(node)
                    del node_sequences[wave]

        # А теперь удаляем все символы занесенные в список deleted_samples_indexes из
        # списка samples.
        reduction = 0
        for i in deleted_samples_indexes:
            i -= reduction
            samples.pop(i)
            reduction +=1

        # Удалим все найденные символы также с картинки:
        cleaned_picture = delete_samples_from_pic(BW_SkeletonWT, samples_to_delete)

        # Здесь надо сформировать словарь key_nodes экземпляров node
        key_nodes_dict = {}
        node_id = 0
        for coordinates in key_nodes:

            # Узлы будем нумеровать, начиная с "1", чтобы не путать метки посещенных
            # пикселей с пикселями, которые еще не посещались ни одной волной ни из
            # одного узла.
            node_id += 1
            key_nodes_dict[node_id] = coordinates

        # Теперь сформируем словарь узлов, которые надо проверить, и начнем
        # нумеровать узлы в порядке после последнего пронумерованного ключевого
        # узла:
        nodes_to_check_dict = {}
        for coordinates in nodes_to_check:
            node_id += 1
            nodes_to_check_dict[node_id] = coordinates

        node_vectors = node_guide_vectors_calc(cleaned_picture, key_nodes_dict,
                                               nodes_to_check_dict)
        key_nodes = create_nodes(key_nodes_dict, node_vectors)

        # Теперь проверим все узлы, которые необходимо проверить, на очищенной
        # картинке, и сформируем окончательные списки ключевых узлов и узлов
        # перегибов:
        additional_key_nodes, bending_nodes = check_nodes(cleaned_picture,
                                                          nodes_to_check_dict,
                                                          key_nodes_dict)
        key_nodes.extend(additional_key_nodes)

        # print('Ключевые узлы на картинке:')
        # for node in key_nodes:
        #     print(node.id, end=', ')
        # print(f'\n')
        # print('Узлы изгибов на картинке:')
        # for node in bending_nodes:
        #     print(node.id, end=', ')
        # print(f'\n')

        # Теперь списки экземпляров узлов с найденными и внесенными для них
        # направляющими векторами передаем в функцию edges_on_the_picture для описания
        # ребер символа, которыми соединяются все его узлы. Эта функция возвращает
        # список экземпляров ребер класса edge.
        edges_list = edges_on_the_picture(cleaned_picture, key_nodes, bending_nodes)

        # Выведем координаты ключевых узлов, соединяемых найденными ребрами:
        # count = 0
        # for _, edge in edges_list:
        #     count += 1
        #     print(f'Ребро номер {count} соединяет ключевые узлы: '
        #           f' {(edge.node_1.i, edge.node_1.j)} и '
        #           f'{(edge.node_2.i, edge.node_2.j)}')

        # Теперь, когда у нас есть список всех ребер на изображении, просканируем
        # картинку, найдем границы символа и увеличим его изображение до размера
        # 90 х 60 пикселей, сохраняя пропорции, т.е. коэфф-т увеличения будем
        # использовать один и тот же для обоих направлений.

        cleaned_picture_matrix = (cleaned_picture / 255).astype('int16')
        i_min, j_min, i_max, j_max = find_sample_bounds(cleaned_picture_matrix)

        # В случае с датасетом CoMNIST не будем изменять размер картинки - он
        # большой, и все картинки меют один и тот же размер:
        resized_edges = resize_struct_pic(NEW_SIZE_I, NEW_SIZE_J, i_min, j_min,
                                          i_max, j_max, edges_list)
        # resized_edges = edges_list

        sample_points = model_struct(resized_edges, NEW_SIZE_I, NEW_SIZE_J)
        # Теперь заполним массив изображения с новым размером изображениями
        # структурных элементов.
        new_struct_pic = np.zeros((NEW_SIZE_I, NEW_SIZE_J))
        for i, j in sample_points:
            # print(f'Значения i = {i}, j = {j}.')
            new_struct_pic[i, j] = 255

        sizeI, sizeJ = new_struct_pic.shape
        pic_with_features = np.zeros((sizeI, sizeJ, 3))
        for i in range(3):
            pic_with_features[:, :, i] = new_struct_pic.copy()

        # Эта строчка нужна при сборе признаков по методу пересечений
        sample_features, cross_coordinates = find_lines_crosses(new_struct_pic,
                                                                probe_lines)

        # Эта строчка нужна при сборе признаков в виде среднего числа пикселей,
        # лежащих на главных диагоналях выделенного участка изображения:
        # sample_features = diagonal_features(new_struct_pic)

        # Эта строчка нужна при сборе признаков в виде среднего числа пикселей,
        # лежащих на строках выделенного участка изображения:
        # sample_features = horizontal_features(new_struct_pic)

        # При сборе признаков в виде среднего числа пикселей на диагоналях или
        # строчках этот участок кода не нужен:
        # for i, j in cross_coordinates:
        #     pic_with_features[i, j, 0] = 255
        #     pic_with_features[i, j, 1] = 0
        #     pic_with_features[i, j, 2] = 0

        # print(f'Из функции пришел список sample_features спризнаками для данного изображения:\n {sample_features}')

        letter_features_add = pd.DataFrame([sample_features])
        letter_features_add['letter'] = letter
        # print(f'Добавляем к общей таблице признаков картинок данной буквы признки для данной картинки: \n {letter_features_add}')
        letter_features = pd.concat([letter_features, letter_features_add],
                                    ignore_index = True)

        # sys.exit(0)
    ##################################################
        # Посмотрим на изображение символа и выделим на нем крайние точки и найденный
        # ноль:
        # crop_center_pic = sample_bounds_pic(cleaned_picture, i_min, j_min,
        #                                     i_max, j_max)

        # Изобразим на скелетизированном очищенном изображении все ребра, которые были
        # найдены:
        # edges_picture = skeleton_edges_pic(cleaned_picture, edges_list)

        "Обозначим на очищенной скелетизированной картинке ключевые узлы и изгибы."
        # colored_cleaned_picture = sample_nodes_pic(cleaned_picture, key_nodes,
        #                                            bending_nodes)

        # Отметим на скелетизированном изображении найденные источники волн:
        # BW_SkeletonWT_colored = wave_sources_pic(BW_SkeletonWT, sources)

        # Отобразим ключевые узлы и узлы-перегибы на скелетизированном изображении:
        # Marked_Skeleton = key_bending_nodes_pic(BW_SkeletonWT, key_nodes,
        #                                          nodes_to_check)

        # Отметим на скелетизированном изображении все найденные на этом изображении
        # символы:
        # samplePicture = all_samples_pic(BW_SkeletonWT, samples, node_sequences)

        "Display the results"
        # ax1, ax2 = ax.ravel()
        # images = [[original_img, BW_Original, BW_SkeletonZS, BW_SkeletonWT_colored,
        #           Marked_Skeleton], [samplePicture, colored_cleaned_picture,
        #           edges_picture, crop_center_pic, new_struct_pic]]   #,
        #           # pic_with_features]]   -  это изображение нужно в случае сбора признаков по методу пересечений.
        # names = [['Original picture', 'Original binary image', 'Skeleton after Zhang-Suen',
        #          'Skeleton after Wu-Tsai', 'Marked key and questionable nodes'],
        #          ['Samples found on the image', 'Cleaned picture with nodes',
        #          'Marked edges', 'Max, Min, Center', 'Model struct']]   #,
        #         #  'Sample features']]    -  это название нужно в случае сбора признаков по методу пересечений.
        # number_of_rows = len(images)
        # for i in range(number_of_rows):
        #     number_of_columns = len(images[i])
        #     figure, plots = plt.subplots(ncols=number_of_columns, nrows=1,
        #                                  figsize=(20, 10))
        #     for pic, name, subplot in zip(images[i], names[i], plots):
        #         subplot.imshow(pic, cmap='gray')
        #         subplot.set_title(name)
        #         subplot.set_axis_off()
        #     plt.show()

        # cv2.imwrite(path, thinned_binary)

    return letter_features, number_of_files

In [ ]:
# Функция построения участков изображения для сбора признаков в виде среднего
# количества пикселей символа в пиксельных строках в этом участке.

def horizontal_features(img):

    # Размеры участка разбиения изображения:
    PIECE_I = 4
    PIECE_J = 8
    # img = (img / 255).astype('int16')
    sample_features = []

    for i_start in range(5, 85, 4):
        for j_start in range(2, 58, 8):

            rows = []

            i_end = i_start + PIECE_I
            j_end = j_start + PIECE_J
            for i in range(i_start, i_end):
                pixels_num = 0
                for j in range(j_start, j_end):
                    if img[i, j] == 255:
                        pixels_num += 1
                rows.append(pixels_num)
            average_pixels_num = round(sum(rows) / len(rows))
            sample_features.append(average_pixels_num)

    return sample_features

In [ ]:
# При смене метода сбора информации необходимо поменять определенные строчки в
# этом основном ячейке и в ячейке с функцией collect_letters_features.
# При изменении датасета на CoMNIST также нужно поменять определенные строчки в
# этом основном ячейке и в ячейке с функцией collect_letters_features.
# Строчки, которые нужно изменять, имеют соответствующие комментарии.

train = ['/content/contentgdrive/MyDrive/Letters']
        # '/content/contentgdrive/MyDrive/Сyrillic_capital']

test = []
train_test_sets = [train]  #, test]

letters_features = pd.DataFrame()

# Эти 2 строки нужны при работе с датасетом строчных русских букв
NEW_SIZE_I = 90
NEW_SIZE_J = 60
# Для случая датасета CoMNIST новые размеры изображения сделаем такими же как и
# старые:
# NEW_SIZE_I = 278
# NEW_SIZE_J = 278

# Эта строчка нужна при сборе признаков методом пересечений:
probe_lines = secant_generator(NEW_SIZE_I, NEW_SIZE_J)
# При сборе признаков по среднему количеству пикселей на главных диагоналях будем
# отправлять в функцию collect_letters_features пустой словарь probe_lines:
# probe_lines = {}

start_time = time.time()

for letters_sets in train_test_sets:
    # folder_path = os.path.join(DIRECTORY, letters_set)
    print(f'Папка {letters_sets}.')
    for folder in letters_sets:
        letters = os.listdir(folder)
        print(f'В этой папке содержатся папки с изображениями следующих букв: '
              f'{letters}.')
        number_of_files = 0
        for letter in ['o']: #letters:
            # files_path = os.path.join(folder, letter)
            print(f'Заходим в папку с изображениями буквы {letter}, '
                  f'адрес которой: {files_path}')
            letters_features_add, number_of_files_add = \
                        collect_letters_features(files_path, letter, probe_lines,
                                                 NEW_SIZE_I, NEW_SIZE_J)
            letters_features = pd.concat([letters_features, letters_features_add],
                                         ignore_index = True)
            number_of_files += number_of_files_add

end_time = time.time()
work_time = end_time - start_time
print(f'На обработку {number_of_files} файлов с изображениями букв {letters} '
      f'было затрачено {work_time} сек.')

In [ ]:
letters_features.to_csv('/content/contentgdrive/MyDrive/Диплом/Data/train/Letter2_crosses.csv')